# Make a FisherMatrix object that was only limited by l_max also have a k_max limit - use linear P(k) versions, one at a time

In [1]:
import crosspower as cp
import FisherCl as fcl # branch quickCl
#import noiseCl as ncl
#import camb

# set plotting to be in the notebook instead of its own qt window
%matplotlib inline

IMPORT ERROR: /opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/quicklens/mpi.pyc (No module named pypar). Could not load pbs or pypar. MPI will not be used.


# Load two k,g + T,E Fisher Objects with noise

In [2]:
# Load saved Fobj2, Fobj4
# The suffix 2 is for nonlinear, suffix 4 is for linear (just to keep with the established pattern)

import FisherCl as fcl
import noiseCl as ncl
import crosspower as cp
import pickle

fileNum = 2

if fileNum == 1:
    filehandler_lin = open('Fisher_7_16O_wa1_median_B.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
elif fileNum == 2:
    filehandler_lin = open('Fisher_7_16O_wa1_median_shapeless_B.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
elif fileNum == 3:
    filehandler_lin = open('Fisher_7_16O_wa1_median_pzkfixed_B.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
elif fileNum == 4:
    filehandler_lin = open('Fisher_7_16O_wa1_median_chifixed_B.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
elif fileNum == 5:
    filehandler_lin = open('Fisher_7_16O_wa1_median_chiShapefixed_B.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,

# need to load another one in order to get the dClkk/dthetas

if fileNum == 1:
    filehandler4 = open('Fisher_7_16O_wa1_median.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
elif fileNum == 2:
    filehandler4 = open('Fisher_7_16O_wa1_median_shapeless.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
elif fileNum == 3:
    filehandler4 = open('Fisher_7_16O_wa1_median_pzkfixed.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
elif fileNum == 4:
    filehandler4 = open('Fisher_7_16O_wa1_median_chifixed.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
elif fileNum == 5:
    filehandler4 = open('Fisher_7_16O_wa1_median_chiShapefixed.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,



Fobj2 = pickle.load(filehandler_lin) 
Fobj4 = pickle.load(filehandler4) 

#label2 = 'Gold nln'
#label4 = 'Opti nln'
#label2 = 'Gold lin'
#label4 = 'Opti lin'

# let Fobj2 control the notebook parameters
lmin = Fobj2.lmin
lmax = Fobj2.lmax
nBins = Fobj2.nBins
nMaps = Fobj2.nBins + 1
nParams = Fobj2.nParams
nCosParams = Fobj2.nParams - Fobj2.nBins
ells = Fobj2.ells
obsList = Fobj2.obsList
obsNames = Fobj2.obsNames
paramList = Fobj2.paramList

print 'nParams: ',nParams
print 'nCosParams: ',nCosParams
print 'obsList: ',obsList
print 'obsNames: ',obsNames
print 'paramList: ',paramList

print 'Fij_kg.shape: ',Fobj2.Fij.shape
print 'Fij_TE.shape: ',Fobj2.FijTE.shape

print 'Fobj4.dClVecs.shape: ', Fobj4.dClVecs.shape

nParams:  25
nCosParams:  9
obsList:  [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.
  29.  30.  31.  32.  33.  36.  37.  38.  39.  40.  41.  42.  43.  44.
  45.  46.  47.  48.  49.  50.  54.  55.  56.  57.  58.  59.  60.  61.
  62.  63.  64.  65.  66.  67.  72.  73.  74.  75.  76.  77.  78.  79.
  80.  81.  82.  83.  84.  90.  91.  92.  93.  94.  95.  96.  97.  98.
  99. 100. 101. 108. 109. 110. 111. 112. 113. 114. 115. 116. 117. 118.
 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 144. 145. 146. 147.
 148. 149. 150. 151. 152. 162. 163. 164. 165. 166. 167. 168. 169. 180.
 181. 182. 183. 184. 185. 186. 198. 199. 200. 201. 202. 203. 216. 217.
 218. 219. 220. 234. 235. 236. 237. 252. 253. 254. 270. 271. 288.]
obsNames:  ['k,k', 'k,g1', 'k,g2', 'k,g3', 'k,g4', 'k,g5', 'k,g6', 'k,g7', 'k,g8', 'k,g9', 'k,g10', 'k,g11', 'k,g12', 'k,g13', 'k,g14', 'k,g15', 'k,g16', 'g1,g1', 'g1,g2', 'g1,g3', 'g1,

In [3]:
print Fobj2.nonlinear,Fobj4.nonlinear

False False


In [4]:
# get the dClkk/dthetas from Fobj4
# kk is the (0,:,:) part
Fobj2.dClVecs[0] = np.copy(Fobj4.dClVecs[0])
filehandler4.close()

print 'done.'

done.


# Create $\chi(z)$ and $\ell_{max}(k_{max})$ functions

In [5]:
# MatterPower has a getChiOfZ function, but MatterPower object has been stripped from saved Fobj
# get params for MatterPower from FisherMatrix:
def getPk(Fobj):
    return cp.MatterPower(nz=Fobj.nz,AccuracyBoost=Fobj.AccuracyBoost,nonlinear=Fobj.nonlinear,**Fobj.cosParams)
def getWin(Fobj,myPk):
    return cp.Window(myPk,zmin=Fobj.zmin,zmax=Fobj.zmax,nBins=Fobj.nBins,
               biasK=cp.ones,biasG=cp.byeBias,dndzMode=Fobj.dndzMode,z0=Fobj.z0,
               doNorm=Fobj.doNorm,useWk=Fobj.useWk,BPZ=Fobj.BPZ,binSmooth=Fobj.binSmooth,
               biasByBin=Fobj.biasByBin)

label2 = 'linear'
print 'get {0} P(k)...'.format(label2)
myPk2 = getPk(Fobj2)
#print 'get {0} P(k)...'.format(label4)
#myPk4 = getPk(Fobj4)

chiOfZ2 = myPk2.getChiofZ()
#chiOfZ4 = myPk4.getChiofZ()

# define bin edges
# the selection of beesBins must be consistent with that which was selected in cp.tophat
beesBins = True
if beesBins:
    if Fobj2.nBins == 6:
      binEdges = [0.0,0.5,1.0,2.0,3.0,4.0,7.0]
    elif Fobj2.nBins == 16:
      binEdges = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 
                  2.3, 2.6, 3.0, 3.5, 4.0, 7.0]
    else:
      print '!!!!! wrong nBins for beesBins detected !!!!!'
      #return 0
else:
    binEdges = np.linspace(Fobj.zmin,Fobj.zmax,Fobj.nBins+1)
print 'bin edges: ',binEdges
    
    
print 'redshift of bin edges: ',binEdges
binCenters = (np.array(binEdges[1:])+np.array(binEdges[:-1]))/2
print 'redshift of bin centers: ',binCenters

print 'Chi of bin edges ({0}): '.format(label2),chiOfZ2(binEdges)
#print 'Chi of bin edges ({0}): '.format(label4),chiOfZ4(binEdges)

print 'Chi of bin centers ({0}): '.format(label2),chiOfZ2(binCenters)
#print 'Chi of bin centers ({0}): '.format(label4),chiOfZ4(binCenters)


get linear P(k)...
neutrino_hierarchy =  degenerate
starting makePkInterp.


finishing makePkInterp.
bin edges:  [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.3, 2.6, 3.0, 3.5, 4.0, 7.0]
redshift of bin edges:  [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.3, 2.6, 3.0, 3.5, 4.0, 7.0]
redshift of bin centers:  [0.1  0.3  0.5  0.7  0.9  1.1  1.3  1.5  1.7  1.9  2.15 2.45 2.8  3.25
 3.75 5.5 ]
Chi of bin edges (linear):  [   0.          859.2189536  1622.3619519  2296.51298368 2891.5804514
 3418.06507734 3885.81232359 4303.48886402 4678.48026482 5016.97390771
 5324.12211493 5735.2395798  6096.86290375 6517.06735343 6964.00877968
 7343.78066275 8823.02783612]
Chi of bin centers (linear):  [ 441.87369959 1252.45615626 1969.97722686 2603.27066668 3162.75623188
 3658.70853466 4100.41888972 4495.90678816 4851.94402391 5174.17997278
 5536.51771533 5921.62911487 6314.91649388 6750.10014948 7161.22576349
 8210.90722966]


In [6]:
# optionally plot the window functions

doTest = False
if doTest:
    # get window object for testing
    myWin2 = getWin(Fobj2,myPk2)

    print myWin2.zmin,myWin2.zmax,myWin2.nBins,myWin2.dndzMode

    # plot galaxy windows
    zs = np.linspace(0,7,1000)
    for binNum in range(1,myWin2.nBins+1):
        galWinFunc = cp.Window.galaxies(myWin2,binNum)
        win = galWinFunc(zs)
        plt.plot(zs,win)
    plt.xlim([0,7])
    #plt.xlim([1,2])
    plt.show()

    # plot kappa windows
    #zs = np.linspace(0,7,1000)
    for binNum in range(1,myWin2.nBins+1):
        kapWinFunc = cp.Window.kappa(myWin2,binNum)
        win = kapWinFunc(zs)
        plt.plot(zs,win)
    plt.xlim([0,7])
    #plt.xlim([1,2])
    plt.show()


# Break apart covar and dClVecs as observables drop out

In [7]:
# set the kmax values used throughout

#kmax1 = 0.05  # don't go below this or bin 2 will have less than 50 ells and crash program
#kmax2 = 0.075

#kmax1 = 0.1
#kmax2 = 0.2

#kmax1 = 1.0
#kmax2 = 2.0  # no need to go above this since it is high enough to only kill bin1

# write them in terms of parameter h
hubble = Fobj2.H0/100
print 'h = ',hubble

kmax2 = 0.3*hubble
kmax1 = kmax2*0.5
print 'kmax1 = ',kmax1
print 'kmaz2 = ',kmax2

h =  0.660068024637
kmax1 =  0.0990102036956
kmaz2 =  0.198020407391


In [8]:
# from getKLimited (below), modified for bin centers and chiOfZ2:
# create lmax function via the Limber approximation
myLmax = lambda kmax: np.floor(kmax*chiOfZ2(binCenters)-0.5).astype(int)
lmaxByBin1 = myLmax(kmax1)
lmaxByBin2 = myLmax(kmax2)

print 'lmaxByBin1: ',lmaxByBin1
print 'lmaxByBin2: ',lmaxByBin2

lmaxByBin1:  [ 43 123 194 257 312 361 405 444 479 511 547 585 624 667 708 812]
lmaxByBin2:  [  87  247  389  515  625  723  811  889  960 1024 1095 1172 1249 1336
 1417 1625]


In [9]:
# above 30 in lowest bin in both cases.  Great.

# Define function for creating blocks for making Fij

In [10]:
# some helper functions for getKlimited, getKZlimited

# define row,col deleting function
def fixParam(Fij,pNum):
    return np.delete(np.delete(Fij,pNum,0),pNum,1)

# define the covar cutting function
def covarCut(covar,dClVecs,preMask,cutNum,verbose=True):
    """
    Purpose:
        produce covar and dClVecs with fewer observables
    Inputs:
        covar,dClVecs: like in Fobj
        preMask: this is a 1d array of binNumbers, 
          indicating which is the lowest bin number used for each row, col of covar 
          and each dClVec
        cutNum: a number contained in preMask indicating which rows, cols of covar, dClVecs to cut
    Returns:
        cut versions of covar,dClVecs,preMask
    """
    #toCut = np.flip(np.where(preMask[0]==cutNum),1)
    toCut = np.flip(np.where(preMask==cutNum),1) # removed 2nd dim of preMask since not needed
    if verbose:
        print 'to cut: ',toCut
    for cut in toCut:
        covar = fixParam(covar,cut)
        #preMask = fixParam(preMask,cut)
        preMask = np.delete(preMask,cut,0)
        dClVecs = np.delete(dClVecs,cut,0)
    if verbose:
        print 'covar.shape: ',covar.shape,', dClVecs.shape: ',dClVecs.shape
        print 'premask: ',preMask
    return covar,dClVecs,preMask


In [11]:
# Define function for creating blocks for making Fij
def getKlimited(binEdges,chiOfZ,Fobj,kmax=0.1,kgLmax=2000,centers=True,galaxiesOnly=False):
    """
    Purpose:
        Chop up Fobj.covar and Fobj.dClVecs into blocks that correspond to having ell values
          below ell(k_max,low_z_bin_edge) in Limber approximation
    Inputs:
        binEdges: an array of redshifts indicating the edges of the redshift bins
        chiOfZ: a distance-redshift relation function chi(z)
        Fobj: FisherMatrix object
        kmax: set the limit that controls the lmax of each redshift bin via Limber appx
          default: 0.1
        kgLmax: ell_max value for k,g
          default: 2000
        centers: set to True to indicate that the values in binEdges are actually bin centers
          default: True
        galaxiesOnly: set to True to cut out the kk, kg parts and return gg only
          default: False
    Uses:
        fixParam: defined above
        covarCut: defined above
    Returns:
        covarBins,dClVecsBins: lists of covar and dClVecs arrays to be used for making Fisher matrix
        lmaxByBin: the ell value for each ell-block edge 
          There will be nBins+2 if centers==True, else nBins+1
        preMaskBins: list of one dimensional masks containing minimum bin numbers for covarBins,dClVecsBins
    """
    # create lmax function via the Limber approximation
    myLmax = lambda kmax: np.floor(kmax*chiOfZ(binEdges)-0.5).astype(int)
    lmaxByBin = myLmax(kmax) # gives array of lmax at lower bin edges (or centers)
    # the -1 is from the -0.5 in the Limber approximation put into the floor function

    # modify this for use in looping below
    lmin = Fobj.lmin
    if centers: # lmaxByBin elements are at bin centers; need to add endpoints for blocks
        lmaxByBin = np.append(Fobj2.lmin,np.append(lmaxByBin,kgLmax))
    else: # lmaxByBin elements are at bin edges, use these for endpoints for blocks
        lmaxByBin[0] = Fobj.lmin
        if lmaxByBin[-1] < kgLmax: 
            lmaxByBin[-1] = kgLmax  # this extends the last bin out to lmax, and this bin is kappa-only
            # >= kgLmax case taken care of below
    print 'kmax = ',kmax,': lmax = ',lmaxByBin
    
    # get codes describing observables
    nBins = Fobj.nBins
    nMaps = Fobj.nBins+1
    print 'obs names: ',Fobj.obsNames
    #print Fobj.obsList
    map1List = np.floor(Fobj.obsList/nMaps)
    map2List = Fobj.obsList%nMaps
    print 'map i numbers: ',map1List
    print 'map j numbers: ',map2List

    print 'invCov shape: ',Fobj.invCov.shape, 'number of observables: ',Fobj.obsList.shape[0]
    print 'lmin, lmax for k,g summation: ',lmin,kgLmax

    # create mask for covariance matrix to be applied to implement lmax
    mSize = Fobj.obsList.__len__()
    preMask = np.zeros((mSize,mSize)) # contains lowest bin number (>0) of 4 used in covar
    print 'preMask shape: ',preMask.shape
    for i in range(mSize):
        for j in range(i,mSize):
            myMaps = np.array([map1List[i],map1List[j],map2List[i],map2List[j]])        
            if i>0 or j>0:
                preMask[i,j] = np.min(myMaps[np.where(myMaps>0)])
                preMask[j,i] = preMask[i,j]
            else:
                preMask[0,0] = 0
            #print i,j,myMaps, myMaps[np.where(myMaps>0)], preMask[i,j]
    preMask = preMask.astype(int)
    print preMask#[0]
    
    covar=Fobj.covar
    dClVecs=Fobj.dClVecs
    preMaskLine = preMask[0] # modified to use just the first line
    
    verbose = True
    if galaxiesOnly:
        # kk and kg is at the beginning of each
        # cut all kg now, ignore kk at the end
        for cutNum in range(nBins):
            #covar,dClVecs,preMaskLine = covarCut(covar,dClVecs,preMaskLine,1,verbose=verbose)
            covar = fixParam(covar,1)
            preMaskLine = np.delete(preMaskLine,1,0)
            dClVecs = np.delete(dClVecs,1,0)
        #covar = covar[nBins+1:,nBins+1:]
        #dClVecs = dClVecs[nBins+1:]
        #preMask = preMask[nBins+1:,nBins+1:]
        print 'galaxiesOnly = True: kappa-g has been removed.'
    print 'preMaskLine: ',preMaskLine
    print 'covar shape: ',covar.shape
    print 'dClVecs shape: ',dClVecs.shape
        
    # cut out all covar,dClVecs,preMask elements with row/col of 1s below lmaxByBin[0] = -1 : so all the way to lmin.

    # prepare containers for the results
    covarBins = []
    dClVecsBins = []
    preMaskBins = []

    # iterate as bins are removed
    myEllMin = lmaxByBin[0]
    myEllMax = lmaxByBin[1]
    lastBlock = False # for stopping iteration if myEllMax is below kgLmax
    if centers:
        nBlocks = nBins+1
    else: 
        nBlocks = nBins
        
    for blockNum in range(nBlocks): 
        print '\nstarting blockNum {0} of {1}...'.format(blockNum+1,nBlocks)
        
        if not centers: # cut bins not needed for this block
            covar,dClVecs,preMaskLine = covarCut(covar,dClVecs,preMaskLine,blockNum+1,verbose=verbose)

        if lmaxByBin[blockNum+1] < kgLmax:
            myEllMin = lmaxByBin[blockNum]
            myEllMax = lmaxByBin[blockNum+1]
        else:
            if lmaxByBin[blockNum] < kgLmax:
                # prob. don't need this if here since only one endpoint will pass kgLmax then end the for loop
                myEllMin = lmaxByBin[blockNum]
            myEllMax = kgLmax
            lastBlock = True
            
        # create block-out
        if galaxiesOnly:  # 1: to remove kappa-kappa
            covarBins.append( covar[1:,1:,myEllMin-lmin:myEllMax-lmin] )
            dClVecsBins.append( dClVecs[1:,:,myEllMin-lmin:myEllMax-lmin] )
            preMaskBins.append( preMaskLine[1:].copy() )
        else:
            covarBins.append( covar[:,:,myEllMin-lmin:myEllMax-lmin] )
            dClVecsBins.append( dClVecs[:,:,myEllMin-lmin:myEllMax-lmin] )
            preMaskBins.append( preMaskLine.copy() )
        if galaxiesOnly and blockNum == nBlocks-2:
            print 'omitting kappa block.'
            lastBlock = True
        
        if centers and blockNum != nBins: # this cut has to be done after block-out
            covar,dClVecs,preMaskLine = covarCut(covar,dClVecs,preMaskLine,blockNum+1,verbose=verbose)
            
        print 'blockNum = ',blockNum,', covarBins.shape = ',covarBins[blockNum].shape,', dClVecsBins.shape = ',dClVecsBins[blockNum].shape
        if verbose:
            print 'covariance shape: ',covarBins[blockNum].shape
            print 'derivative array shape: ',dClVecsBins[blockNum].shape
            print r'bin $\ell_{{min}} = {0}$, bin $\ell_{{max}} = {1}$'.format(lmaxByBin[blockNum],lmaxByBin[blockNum+1]-1)
        if lastBlock:
            break # should break the for blockNum loop
            
    print 'done'  
    return covarBins,dClVecsBins,lmaxByBin,preMaskBins

# test it
#doTest = True
doTest = False
if doTest:
    kmax_t = kmax2
    #kmax_t = 5
    print 'starting nonlinear P(k), kmax = {0}...'.format(kmax_t)
    covarBins2_01,dClVecsBins2_01,lmaxByBin2_01,preMaskBins2_01 = getKlimited(binEdges,chiOfZ2,Fobj2,kmax=kmax_t,
                                                                              centers=False)
    #covarBins2_01,dClVecsBins2_01,lmaxByBin2_01,preMaskBins2_01 = getKlimited(binCenters,chiOfZ2,Fobj2,kmax=kmax_t,
    #                                                                          centers=True)
    covarBins2_01g,dClVecsBins2_01g,lmaxByBin2_01g,preMaskBins2_01g = getKlimited(binEdges,chiOfZ2,Fobj2,kmax=kmax_t,
                                                                              centers=False,galaxiesOnly=True)
    #covarBins2_01g,dClVecsBins2_01g,lmaxByBin2_01g,preMaskBins2_01g = getKlimited(binCenters,chiOfZ2,Fobj2,kmax=kmax_t,
    #                                                                          centers=True,galaxiesOnly=True)


In [12]:
if doTest:
    print 'not galaxiesOnly:'
    print dClVecsBins2_01.__len__()
    #print invCovBins2_01.__len__()
    print covarBins2_01.__len__()

    for blockNum in range(dClVecsBins2_01.__len__()):
        print 'blockNum ',blockNum
        print 'dClVecs shape: ',dClVecsBins2_01[blockNum].shape
        #print 'invCovar shape: ',invCovBins2_01[blockNum].shape
        print 'covar shape: ',covarBins2_01[blockNum].shape
        print 'lmaxByBin: ',lmaxByBin2_01[blockNum]
        print 'preMaskBins: ',preMaskBins2_01[blockNum]
        
    print '\ngalaxiesOnly:'
    print dClVecsBins2_01g.__len__()
    #print invCovBins2_01g.__len__()
    print covarBins2_01g.__len__()

    for blockNum in range(dClVecsBins2_01g.__len__()):
        print 'blockNum ',blockNum
        print 'dClVecs shape: ',dClVecsBins2_01g[blockNum].shape
        #print 'invCovar shape: ',invCovBins2_01g[blockNum].shape
        print 'covar shape: ',covarBins2_01g[blockNum].shape
        print 'lmaxByBin: ',lmaxByBin2_01g[blockNum]
        print 'preMaskBins: ',preMaskBins2_01g[blockNum]


In [13]:
# testing
# getSigmas copied from below
def getSigmas(Fij):
    #print np.diag(np.linalg.inv(Fij))
    return np.sqrt(np.diag(np.linalg.inv(Fij)))

if doTest:
    invCovOneC = np.transpose(np.linalg.inv(np.transpose(covarBins2_01[0])))
    dClVecs50C = dClVecsBins2_01[0][:,:,myLmin-lmin:myLmax-lmin]
    invCov50C = invCovOneC[:,:,myLmin-lmin:myLmax-lmin]
    Fij50C = makeFij(dClVecs50C,invCov50C,Fobj2.nParams)
    Fij50C = fixParam(Fij50C,9)
    sigmas50C = getSigmas(Fij50C)

    #print 'without bin1'
    #t = PrettyTable(['Parameter','sigmas_no_bin1B','sigmas_no_bin1C','percent diff.'])
    #for i,p in enumerate(paramList[:nCosParams]):
    #    Zi = sigmas50B[i]
    #    Bi = sigmas50C[i]
    #    diff = (Bi-Zi)/Zi *100
    #    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
    #print t


In [14]:
#centers = True
centers = False
if centers:
    print 'starting {1} P(k), kmax = {0}...'.format(kmax1,label2)
    covarBins2_01,dClVecsBins2_01,lmaxByBin2_01,preMaskBins2_01 = getKlimited(binCenters,chiOfZ2,Fobj2,kmax=kmax1,
                                                                              centers=centers)
    print 'starting {1} P(k), kmax = {0}...'.format(kmax2,label2)
    covarBins2_02,dClVecsBins2_02,lmaxByBin2_02,preMaskBins2_02 = getKlimited(binCenters,chiOfZ2,Fobj2,kmax=kmax2,
                                                                              centers=centers)

    #print 'starting {1} P(k), kmax = {0}...'.format(kmax1,label4)
    #covarBins4_01,dClVecsBins4_01,lmaxByBin4_01,preMaskBins4_01 = getKlimited(binCenters,chiOfZ4,Fobj4,kmax=kmax1,
    #                                                                          centers=centers)
    #print 'starting {1} P(k), kmax = {0}...'.format(kmax2,label4)
    #covarBins4_02,dClVecsBins4_02,lmaxByBin4_02,preMaskBins4_02 = getKlimited(binCenters,chiOfZ4,Fobj4,kmax=kmax2,
    #                                                                          centers=centers)
else:
    print 'starting {1} P(k), kmax = {0}...'.format(kmax1,label2)
    covarBins2_01,dClVecsBins2_01,lmaxByBin2_01,preMaskBins2_01 = getKlimited(binEdges,chiOfZ2,Fobj2,kmax=kmax1,
                                                                              centers=centers)
    print 'starting {1} P(k), kmax = {0}...'.format(kmax2,label2)
    covarBins2_02,dClVecsBins2_02,lmaxByBin2_02,preMaskBins2_02 = getKlimited(binEdges,chiOfZ2,Fobj2,kmax=kmax2,
                                                                              centers=centers)

    #print 'starting {1} P(k), kmax = {0}...'.format(kmax1,label4)
    #covarBins4_01,dClVecsBins4_01,lmaxByBin4_01,preMaskBins4_01 = getKlimited(binEdges,chiOfZ4,Fobj4,kmax=kmax1,
    #                                                                          centers=centers)
    #print 'starting {1} P(k), kmax = {0}...'.format(kmax2,label4)
    #covarBins4_02,dClVecsBins4_02,lmaxByBin4_02,preMaskBins4_02 = getKlimited(binEdges,chiOfZ4,Fobj4,kmax=kmax2,
    #                                                                          centers=centers)
    
print 'done with all 2 cases.'

starting linear P(k), kmax = 0.0990102036956...
kmax =  0.0990102036956 : lmax =  [   2   84  160  226  285  337  384  425  462  496  526  567  603  644
  689  726 2000]
obs names:  ['k,k', 'k,g1', 'k,g2', 'k,g3', 'k,g4', 'k,g5', 'k,g6', 'k,g7', 'k,g8', 'k,g9', 'k,g10', 'k,g11', 'k,g12', 'k,g13', 'k,g14', 'k,g15', 'k,g16', 'g1,g1', 'g1,g2', 'g1,g3', 'g1,g4', 'g1,g5', 'g1,g6', 'g1,g7', 'g1,g8', 'g1,g9', 'g1,g10', 'g1,g11', 'g1,g12', 'g1,g13', 'g1,g14', 'g1,g15', 'g1,g16', 'g2,g2', 'g2,g3', 'g2,g4', 'g2,g5', 'g2,g6', 'g2,g7', 'g2,g8', 'g2,g9', 'g2,g10', 'g2,g11', 'g2,g12', 'g2,g13', 'g2,g14', 'g2,g15', 'g2,g16', 'g3,g3', 'g3,g4', 'g3,g5', 'g3,g6', 'g3,g7', 'g3,g8', 'g3,g9', 'g3,g10', 'g3,g11', 'g3,g12', 'g3,g13', 'g3,g14', 'g3,g15', 'g3,g16', 'g4,g4', 'g4,g5', 'g4,g6', 'g4,g7', 'g4,g8', 'g4,g9', 'g4,g10', 'g4,g11', 'g4,g12', 'g4,g13', 'g4,g14', 'g4,g15', 'g4,g16', 'g5,g5', 'g5,g6', 'g5,g7', 'g5,g8', 'g5,g9', 'g5,g10', 'g5,g11', 'g5,g12', 'g5,g13', 'g5,g14', 'g5,g15', 'g5,g16', 'g6,g6', '

covar.shape:  (45, 45, 2999) , dClVecs.shape:  (45, 25, 2999)
premask:  [ 0  9 10 11 12 13 14 15 16  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10
 11 11 11 11 11 11 12 12 12 12 12 13 13 13 13 14 14 14 15 15 16]
blockNum =  7 , covarBins.shape =  (45, 45, 37) , dClVecsBins.shape =  (45, 25, 37)
covariance shape:  (45, 45, 37)
derivative array shape:  (45, 25, 37)
bin $\ell_{min} = 425$, bin $\ell_{max} = 461$

starting blockNum 9 of 16...
to cut:  [[16 15 14 13 12 11 10  9  1]]
covar.shape:  (36, 36, 2999) , dClVecs.shape:  (36, 25, 2999)
premask:  [ 0 10 11 12 13 14 15 16 10 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12
 12 12 13 13 13 13 14 14 14 15 15 16]
blockNum =  8 , covarBins.shape =  (36, 36, 34) , dClVecsBins.shape =  (36, 25, 34)
covariance shape:  (36, 36, 34)
derivative array shape:  (36, 25, 34)
bin $\ell_{min} = 462$, bin $\ell_{max} = 495$

starting blockNum 10 of 16...
to cut:  [[14 13 12 11 10  9  8  1]]
covar.shape:  (28, 28, 2999) , dClVecs.shape:  (28, 25, 2999)
prema

covar.shape:  (105, 105, 2999) , dClVecs.shape:  (105, 25, 2999)
premask:  [ 0  4  5  6  7  8  9 10 11 12 13 14 15 16  4  4  4  4  4  4  4  4  4  4
  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6
  6  6  7  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9
  9  9  9  9  9 10 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12 12 12 13
 13 13 13 14 14 14 15 15 16]
blockNum =  2 , covarBins.shape =  (105, 105, 134) , dClVecsBins.shape =  (105, 25, 134)
covariance shape:  (105, 105, 134)
derivative array shape:  (105, 25, 134)
bin $\ell_{min} = 320$, bin $\ell_{max} = 453$

starting blockNum 4 of 16...
to cut:  [[26 25 24 23 22 21 20 19 18 17 16 15 14  1]]
covar.shape:  (91, 91, 2999) , dClVecs.shape:  (91, 25, 2999)
premask:  [ 0  5  6  7  8  9 10 11 12 13 14 15 16  5  5  5  5  5  5  5  5  5  5  5
  5  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11
 11 11 11 11 12 1

In [15]:
# same for galaxies-only case:
if centers:
    print 'starting {1} P(k), kmax = {0}...'.format(kmax1,label2)
    covarBins2_01g,dClVecsBins2_01g,lmaxByBin2_01g,preMaskBins2_01g = getKlimited(binCenters,chiOfZ2,Fobj2,kmax=kmax1,
                                                                              centers=centers,galaxiesOnly=True)
    print 'starting {1} P(k), kmax = {0}...'.format(kmax2,label2)
    covarBins2_02g,dClVecsBins2_02g,lmaxByBin2_02g,preMaskBins2_02g = getKlimited(binCenters,chiOfZ2,Fobj2,kmax=kmax2,
                                                                              centers=centers,galaxiesOnly=True)

    #print 'starting {1} P(k), kmax = {0}...'.format(kmax1,label4)
    #covarBins4_01g,dClVecsBins4_01g,lmaxByBin4_01g,preMaskBins4_01g = getKlimited(binCenters,chiOfZ4,Fobj4,kmax=kmax1,
    #                                                                          centers=centers,galaxiesOnly=True)
    #print 'starting {1} P(k), kmax = {0}...'.format(kmax2,label4)
    #covarBins4_02g,dClVecsBins4_02g,lmaxByBin4_02g,preMaskBins4_02g = getKlimited(binCenters,chiOfZ4,Fobj4,kmax=kmax2,
    #                                                                          centers=centers,galaxiesOnly=True)
else:
    print 'starting {1} P(k), kmax = {0}...'.format(kmax1,label2)
    covarBins2_01g,dClVecsBins2_01g,lmaxByBin2_01g,preMaskBins2_01g = getKlimited(binEdges,chiOfZ2,Fobj2,kmax=kmax1,
                                                                              centers=centers,galaxiesOnly=True)
    print 'starting {1} P(k), kmax = {0}...'.format(kmax2,label2)
    covarBins2_02g,dClVecsBins2_02g,lmaxByBin2_02g,preMaskBins2_02g = getKlimited(binEdges,chiOfZ2,Fobj2,kmax=kmax2,
                                                                              centers=centers,galaxiesOnly=True)

    #print 'starting {1} P(k), kmax = {0}...'.format(kmax1,label4)
    #covarBins4_01g,dClVecsBins4_01g,lmaxByBin4_01g,preMaskBins4_01g = getKlimited(binEdges,chiOfZ4,Fobj4,kmax=kmax1,
    #                                                                          centers=centers,galaxiesOnly=True)
    #print 'starting {1} P(k), kmax = {0}...'.format(kmax2,label4)
    #covarBins4_02g,dClVecsBins4_02g,lmaxByBin4_02g,preMaskBins4_02g = getKlimited(binEdges,chiOfZ4,Fobj4,kmax=kmax2,
    #                                                                          centers=centers,galaxiesOnly=True)
    
print 'done with all 2 cases.'

starting linear P(k), kmax = 0.0990102036956...
kmax =  0.0990102036956 : lmax =  [   2   84  160  226  285  337  384  425  462  496  526  567  603  644
  689  726 2000]
obs names:  ['k,k', 'k,g1', 'k,g2', 'k,g3', 'k,g4', 'k,g5', 'k,g6', 'k,g7', 'k,g8', 'k,g9', 'k,g10', 'k,g11', 'k,g12', 'k,g13', 'k,g14', 'k,g15', 'k,g16', 'g1,g1', 'g1,g2', 'g1,g3', 'g1,g4', 'g1,g5', 'g1,g6', 'g1,g7', 'g1,g8', 'g1,g9', 'g1,g10', 'g1,g11', 'g1,g12', 'g1,g13', 'g1,g14', 'g1,g15', 'g1,g16', 'g2,g2', 'g2,g3', 'g2,g4', 'g2,g5', 'g2,g6', 'g2,g7', 'g2,g8', 'g2,g9', 'g2,g10', 'g2,g11', 'g2,g12', 'g2,g13', 'g2,g14', 'g2,g15', 'g2,g16', 'g3,g3', 'g3,g4', 'g3,g5', 'g3,g6', 'g3,g7', 'g3,g8', 'g3,g9', 'g3,g10', 'g3,g11', 'g3,g12', 'g3,g13', 'g3,g14', 'g3,g15', 'g3,g16', 'g4,g4', 'g4,g5', 'g4,g6', 'g4,g7', 'g4,g8', 'g4,g9', 'g4,g10', 'g4,g11', 'g4,g12', 'g4,g13', 'g4,g14', 'g4,g15', 'g4,g16', 'g5,g5', 'g5,g6', 'g5,g7', 'g5,g8', 'g5,g9', 'g5,g10', 'g5,g11', 'g5,g12', 'g5,g13', 'g5,g14', 'g5,g15', 'g5,g16', 'g6,g6', '

covar.shape:  (22, 22, 2999) , dClVecs.shape:  (22, 25, 2999)
premask:  [ 0 11 11 11 11 11 11 12 12 12 12 12 13 13 13 13 14 14 14 15 15 16]
blockNum =  9 , covarBins.shape =  (21, 21, 30) , dClVecsBins.shape =  (21, 25, 30)
covariance shape:  (21, 21, 30)
derivative array shape:  (21, 25, 30)
bin $\ell_{min} = 496$, bin $\ell_{max} = 525$

starting blockNum 11 of 16...
to cut:  [[6 5 4 3 2 1]]
covar.shape:  (16, 16, 2999) , dClVecs.shape:  (16, 25, 2999)
premask:  [ 0 12 12 12 12 12 13 13 13 13 14 14 14 15 15 16]
blockNum =  10 , covarBins.shape =  (15, 15, 41) , dClVecsBins.shape =  (15, 25, 41)
covariance shape:  (15, 15, 41)
derivative array shape:  (15, 25, 41)
bin $\ell_{min} = 526$, bin $\ell_{max} = 566$

starting blockNum 12 of 16...
to cut:  [[5 4 3 2 1]]
covar.shape:  (11, 11, 2999) , dClVecs.shape:  (11, 25, 2999)
premask:  [ 0 13 13 13 13 14 14 14 15 15 16]
blockNum =  11 , covarBins.shape =  (10, 10, 36) , dClVecsBins.shape =  (10, 25, 36)
covariance shape:  (10, 10, 36)
d

covar.shape:  (67, 67, 2999) , dClVecs.shape:  (67, 25, 2999)
premask:  [ 0  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11
 11 11 11 11 12 12 12 12 12 13 13 13 13 14 14 14 15 15 16]
blockNum =  4 , covarBins.shape =  (66, 66, 104) , dClVecsBins.shape =  (66, 25, 104)
covariance shape:  (66, 66, 104)
derivative array shape:  (66, 25, 104)
bin $\ell_{min} = 572$, bin $\ell_{max} = 675$

starting blockNum 6 of 16...
to cut:  [[11 10  9  8  7  6  5  4  3  2  1]]
covar.shape:  (56, 56, 2999) , dClVecs.shape:  (56, 25, 2999)
premask:  [ 0  7  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9  9
  9  9  9  9 10 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12 12 12 13 13
 13 13 14 14 14 15 15 16]
blockNum =  5 , covarBins.shape =  (55, 55, 92) , dClVecsBins.shape =  (55, 25, 92)
covariance shape:  (55, 55, 92)
derivative array shape:  (55, 25, 92)
bin $\ell_{min} = 676$, bin $\ell_{max} = 76

In [16]:
# save the results
toSave1 = [covarBins2_01,dClVecsBins2_01,lmaxByBin2_01,preMaskBins2_01,covarBins2_01g,dClVecsBins2_01g,lmaxByBin2_01g,preMaskBins2_01g]
toSave2 = [covarBins2_02,dClVecsBins2_02,lmaxByBin2_02,preMaskBins2_02,covarBins2_02g,dClVecsBins2_02g,lmaxByBin2_02g,preMaskBins2_02g]
#toSave3 = [covarBins4_01,dClVecsBins4_01,lmaxByBin4_01,preMaskBins4_01,covarBins4_01g,dClVecsBins4_01g,lmaxByBin4_01g,preMaskBins4_01g]
#toSave4 = [covarBins4_02,dClVecsBins4_02,lmaxByBin4_02,preMaskBins4_02,covarBins4_02g,dClVecsBins4_02g,lmaxByBin4_02g,preMaskBins4_02g]

#np.save('Klimited_gold_nln_k1_wa1.npy',toSave1)
#np.save('Klimited_gold_nln_k2_wa1.npy',toSave2)
#np.save('Klimited_opti_nln_k1_wa1.npy',toSave3)
#np.save('Klimited_opti_nln_k2_wa1.npy',toSave4)

if fileNum == 1:
    np.save('Klimited_gold_lin_k1_wa1_full_C.npy',toSave1)
    np.save('Klimited_gold_lin_k2_wa1_full_C.npy',toSave2)
elif fileNum == 2:
    np.save('Klimited_gold_lin_k1_wa1_shapeless_C.npy',toSave1)
    np.save('Klimited_gold_lin_k2_wa1_shapeless_C.npy',toSave2)
elif fileNum == 3:
    np.save('Klimited_gold_lin_k1_wa1_pzkfixed_C.npy',toSave1)
    np.save('Klimited_gold_lin_k2_wa1_pzkfixed_C.npy',toSave2)
elif fileNum == 4:
    np.save('Klimited_gold_lin_k1_wa1_chifixed_C.npy',toSave1)
    np.save('Klimited_gold_lin_k2_wa1_chifixed_C.npy',toSave2)
elif fileNum == 5:
    np.save('Klimited_gold_lin_k1_wa1_chiShapefixed_C.npy',toSave1)
    np.save('Klimited_gold_lin_k2_wa1_chiShapefixed_C.npy',toSave2)
    

print 'done.'

done.


# Get inverse covariance matrices

In [17]:
# get inverse covariance matrices
def getInvArray(covarBins):
    invCovBins = []
    nBlocks = covarBins.__len__()
    for binNum in range(nBlocks):
        invCovBins.append( np.transpose(np.linalg.inv(np.transpose(covarBins[binNum]))) )
    return invCovBins

invCovBins2_01 = getInvArray(covarBins2_01)
invCovBins2_02 = getInvArray(covarBins2_02)
#invCovBins4_01 = getInvArray(covarBins4_01)
#invCovBins4_02 = getInvArray(covarBins4_02)

# galaxies only cases
invCovBins2_01g = getInvArray(covarBins2_01g)
invCovBins2_02g = getInvArray(covarBins2_02g)
#invCovBins4_01g = getInvArray(covarBins4_01g)
#invCovBins4_02g = getInvArray(covarBins4_02g)
print 'done'

done


In [18]:
# cut to myLmin in first bin
myLmin = 30
minEllInd = myLmin-Fobj2.lmin

dClVecsBins2_01[0] = dClVecsBins2_01[0][:,:,minEllInd:]
covarBins2_01[0]  =  covarBins2_01[0][:,:,minEllInd:]
invCovBins2_01[0]  =  invCovBins2_01[0][:,:,minEllInd:]
dClVecsBins2_02[0] = dClVecsBins2_02[0][:,:,minEllInd:]
covarBins2_02[0]  =  covarBins2_02[0][:,:,minEllInd:]
invCovBins2_02[0]  =  invCovBins2_02[0][:,:,minEllInd:]

#dClVecsBins4_01[0] = dClVecsBins4_01[0][:,:,minEllInd:]
#covarBins4_01[0]  =  covarBins4_01[0][:,:,minEllInd:]
#invCovBins4_01[0]  =  invCovBins4_01[0][:,:,minEllInd:]
#dClVecsBins4_02[0] = dClVecsBins4_02[0][:,:,minEllInd:]
#covarBins4_02[0]  =  covarBins4_02[0][:,:,minEllInd:]
#invCovBins4_02[0]  =  invCovBins4_02[0][:,:,minEllInd:]

# galaxies only cases

dClVecsBins2_01g[0] = dClVecsBins2_01g[0][:,:,minEllInd:]
covarBins2_01g[0]  =  covarBins2_01g[0][:,:,minEllInd:]
invCovBins2_01g[0]  =  invCovBins2_01g[0][:,:,minEllInd:]
dClVecsBins2_02g[0] = dClVecsBins2_02g[0][:,:,minEllInd:]
covarBins2_02g[0]  =  covarBins2_02g[0][:,:,minEllInd:]
invCovBins2_02g[0]  =  invCovBins2_02g[0][:,:,minEllInd:]

#dClVecsBins4_01g[0] = dClVecsBins4_01g[0][:,:,minEllInd:]
#covarBins4_01g[0]  =  covarBins4_01g[0][:,:,minEllInd:]
#invCovBins4_01g[0]  =  invCovBins4_01g[0][:,:,minEllInd:]
#dClVecsBins4_02g[0] = dClVecsBins4_02g[0][:,:,minEllInd:]
#covarBins4_02g[0]  =  covarBins4_02g[0][:,:,minEllInd:]
#invCovBins4_02g[0]  =  invCovBins4_02g[0][:,:,minEllInd:]

print 'done.'

done.


# Assemble the $F_{ij}$ matrices and add them up

In [19]:
def makeFij(dClVecs,invCov,nParams):
    """
    This may replace the previous makeFisher function... blah blah blah.
    Inputs:
        all must have same ell range.  All ell values will be summed over.
    """
    Fij = np.zeros((nParams,nParams)) # indices match those in paramList
    for i in range(nParams):
      dClVec_i = dClVecs[:,i,:] # shape (nCls,nElls)
      for j in range(nParams):
        dClVec_j = dClVecs[:,j,:] # shape (nCls,nElls)
        # here is the summation over ell:
        for ellInd in range(invCov.shape[2]):
          #print dClVec_i.shape,invCov.shape,dClVec_j.shape
          fij = np.dot(dClVec_i[:,ellInd],np.dot(invCov[:,:,ellInd],dClVec_j[:,ellInd]))
          Fij[i,j] += fij
    return Fij


In [20]:
# assemble the avengers
#def getFijSum(nParams,nBins,dClVecsBins,invCovBins,Fij_kk,paramList):
def getFijSum(nParams,nBins,dClVecsBins,invCovBins,paramList,centers=False):
    """
    Inputs:
    
    Returns:
    
    """
    Fij = np.zeros((nParams,nParams)) # indices match those in paramList
    nBlocks = dClVecsBins.__len__()
    nBlocks2 = invCovBins.__len__()
    print 'nBlocks: ',nBlocks,', nBlocks2: ',nBlocks2
    for binNum in range(nBlocks):
        print binNum, dClVecsBins[binNum].size,dClVecsBins[binNum].shape,invCovBins[binNum].shape
        Fij += makeFij(dClVecsBins[binNum],invCovBins[binNum],nParams)

    # add the low-ell kappa-only part (high ell part was binNum = nBins-1)
    nCosParams = nParams-nBins
    #Fij[:nCosParams,:nCosParams] += Fij_kk
    
    if not centers:
        # cut out bin1 from Fij and paramList since it was never used
        Fij = fixParam(Fij,nCosParams)
        myParamList = np.delete(paramList,nCosParams,0)

    print myParamList
    #print Fij
    return Fij,myParamList

#Fij2_01,paramList2_01 = getFijSum(Fobj2.nParams,Fobj2.nBins,dClVecsBins2_01,invCovBins2_01,Fij_kk2_01,Fobj2.paramList)
#Fij2_02,paramList2_02 = getFijSum(Fobj2.nParams,Fobj2.nBins,dClVecsBins2_02,invCovBins2_02,Fij_kk2_02,Fobj2.paramList)
#Fij4_01,paramList4_01 = getFijSum(Fobj4.nParams,Fobj4.nBins,dClVecsBins4_01,invCovBins4_01,Fij_kk4_01,Fobj4.paramList)
#Fij4_02,paramList4_02 = getFijSum(Fobj4.nParams,Fobj4.nBins,dClVecsBins4_02,invCovBins4_02,Fij_kk4_02,Fobj4.paramList)
Fij2_01,paramList2_01 = getFijSum(Fobj2.nParams,Fobj2.nBins,dClVecsBins2_01,invCovBins2_01,Fobj2.paramList,
                                  centers=centers)
Fij2_02,paramList2_02 = getFijSum(Fobj2.nParams,Fobj2.nBins,dClVecsBins2_02,invCovBins2_02,Fobj2.paramList,
                                  centers=centers)
#Fij4_01,paramList4_01 = getFijSum(Fobj4.nParams,Fobj4.nBins,dClVecsBins4_01,invCovBins4_01,Fobj4.paramList,
#                                  centers=centers)
#Fij4_02,paramList4_02 = getFijSum(Fobj4.nParams,Fobj4.nBins,dClVecsBins4_02,invCovBins4_02,Fobj4.paramList,
#                                  centers=centers)
print 'done.'

nBlocks:  16 , nBlocks2:  16
0 183600 (136, 25, 54) (136, 136, 54)
1 228000 (120, 25, 76) (120, 120, 76)
2 173250 (105, 25, 66) (105, 105, 66)
3 134225 (91, 25, 59) (91, 91, 59)
4 101400 (78, 25, 52) (78, 78, 52)
5 77550 (66, 25, 47) (66, 66, 47)
6 56375 (55, 25, 41) (55, 55, 41)
7 41625 (45, 25, 37) (45, 45, 37)
8 30600 (36, 25, 34) (36, 36, 34)
9 21000 (28, 25, 30) (28, 28, 30)
10 21525 (21, 25, 41) (21, 21, 41)
11 13500 (15, 25, 36) (15, 15, 36)
12 10250 (10, 25, 41) (10, 10, 41)
13 6750 (6, 25, 45) (6, 6, 45)
14 2775 (3, 25, 37) (3, 3, 37)
15 31850 (1, 25, 1274) (1, 1, 1274)
['ombh2' 'omch2' 'cosmomc_theta' 'As' 'ns' 'tau' 'mnu' 'w' 'wa' 'bin2'
 'bin3' 'bin4' 'bin5' 'bin6' 'bin7' 'bin8' 'bin9' 'bin10' 'bin11' 'bin12'
 'bin13' 'bin14' 'bin15' 'bin16']
nBlocks:  16 , nBlocks2:  16
0 472600 (136, 25, 139) (136, 136, 139)
1 453000 (120, 25, 151) (120, 120, 151)
2 351750 (105, 25, 134) (105, 105, 134)
3 268450 (91, 25, 118) (91, 91, 118)
4 202800 (78, 25, 104) (78, 78, 104)
5 151800 (66

In [21]:
# galaxies only cases
Fij2_01g,paramList2_01g = getFijSum(Fobj2.nParams,Fobj2.nBins,dClVecsBins2_01g,invCovBins2_01g,Fobj2.paramList,
                                  centers=centers)
Fij2_02g,paramList2_02g = getFijSum(Fobj2.nParams,Fobj2.nBins,dClVecsBins2_02g,invCovBins2_02g,Fobj2.paramList,
                                  centers=centers)
#Fij4_01g,paramList4_01g = getFijSum(Fobj4.nParams,Fobj4.nBins,dClVecsBins4_01g,invCovBins4_01g,Fobj4.paramList,
#                                  centers=centers)
#Fij4_02g,paramList4_02g = getFijSum(Fobj4.nParams,Fobj4.nBins,dClVecsBins4_02g,invCovBins4_02g,Fobj4.paramList,
#                                  centers=centers)
print 'done.'

nBlocks:  15 , nBlocks2:  15
0 162000 (120, 25, 54) (120, 120, 54)
1 199500 (105, 25, 76) (105, 105, 76)
2 150150 (91, 25, 66) (91, 91, 66)
3 115050 (78, 25, 59) (78, 78, 59)
4 85800 (66, 25, 52) (66, 66, 52)
5 64625 (55, 25, 47) (55, 55, 47)
6 46125 (45, 25, 41) (45, 45, 41)
7 33300 (36, 25, 37) (36, 36, 37)
8 23800 (28, 25, 34) (28, 28, 34)
9 15750 (21, 25, 30) (21, 21, 30)
10 15375 (15, 25, 41) (15, 15, 41)
11 9000 (10, 25, 36) (10, 10, 36)
12 6150 (6, 25, 41) (6, 6, 41)
13 3375 (3, 25, 45) (3, 3, 45)
14 925 (1, 25, 37) (1, 1, 37)
['ombh2' 'omch2' 'cosmomc_theta' 'As' 'ns' 'tau' 'mnu' 'w' 'wa' 'bin2'
 'bin3' 'bin4' 'bin5' 'bin6' 'bin7' 'bin8' 'bin9' 'bin10' 'bin11' 'bin12'
 'bin13' 'bin14' 'bin15' 'bin16']
nBlocks:  15 , nBlocks2:  15
0 417000 (120, 25, 139) (120, 120, 139)
1 396375 (105, 25, 151) (105, 105, 151)
2 304850 (91, 25, 134) (91, 91, 134)
3 230100 (78, 25, 118) (78, 78, 118)
4 171600 (66, 25, 104) (66, 66, 104)
5 126500 (55, 25, 92) (55, 55, 92)
6 93375 (45, 25, 83) (45, 

In [22]:
# get k-unlimited Fij_kg.s
Fij2_00 = Fobj2.makeFisher(myLmin,TE=False)
#Fij4_00 = Fobj4.makeFisher(myLmin,TE=False)
print 'done.'

done.


In [23]:
# check sigmas
def getSigmas(Fij):
    #print np.diag(np.linalg.inv(Fij))
    return np.sqrt(np.diag(np.linalg.inv(Fij)))

sigmas2_00 = getSigmas(Fij2_00)
#sigmas4_00 = getSigmas(Fij4_00)

# these are done later with kg in their names
#sigmas2_01 = getSigmas(Fij2_01)
#sigmas2_02 = getSigmas(Fij2_02)
#sigmas4_01 = getSigmas(Fij4_01)
#sigmas4_02 = getSigmas(Fij4_02)
#
#sigmas2_01g = getSigmas(Fij2_01g)
#sigmas2_02g = getSigmas(Fij2_02g)
#sigmas4_01g = getSigmas(Fij4_01g)
#sigmas4_02g = getSigmas(Fij4_02g)

print 'done.'

done.


In [24]:
# compare general result to specialized one
if doTest:
    sigmas2_01 = getSigmas(Fij2_01)
    print 'without bin1 vs kmin 1'
    t = PrettyTable(['Parameter','sigmas_no_bin1C','sigmas2_01','percent diff.'])
    for i,p in enumerate(paramList[:nCosParams]):
        Zi = sigmas50C[i]
        Bi = sigmas2_01[i]
        diff = (Bi-Zi)/Zi *100
        t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
    print t


In [25]:
# bingo. kmin 1, which removes even more info than just bin 1, shows slightly higher uncertainty.

In [26]:
# add the kappa bit to the Fij50C part and check again
"""
Fij50C = makeFij(dClVecs50C,invCov50C,Fobj2.nParams)
Fij50C[:nCosParams,:nCosParams] += Fij_kk2_01
Fij50C = fixParam(Fij50C,9)
sigmas50C2 = getSigmas(Fij50C)
print 'without bin1'
t = PrettyTable(['Parameter','sigmas_no_bin1C2','sigmas2_01','percent diff.'])
for i,p in enumerate(paramList[:nCosParams]):
    Zi = sigmas50C2[i]
    Bi = sigmas2_01[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t
"""

'\nFij50C = makeFij(dClVecs50C,invCov50C,Fobj2.nParams)\nFij50C[:nCosParams,:nCosParams] += Fij_kk2_01\nFij50C = fixParam(Fij50C,9)\nsigmas50C2 = getSigmas(Fij50C)\nprint \'without bin1\'\nt = PrettyTable([\'Parameter\',\'sigmas_no_bin1C2\',\'sigmas2_01\',\'percent diff.\'])\nfor i,p in enumerate(paramList[:nCosParams]):\n    Zi = sigmas50C2[i]\n    Bi = sigmas2_01[i]\n    diff = (Bi-Zi)/Zi *100\n    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )\nprint t\n'

# define function to get f_ij^kk

In [27]:
# define function to get f_ij^kk, or f_ij^gg
def getFijkk(Fobj,myLmin=50,lmax=2000,verbose=False,gg=0):
    """
        Inputs:
            Fobj: a FisherMatrix object
            myLmin, lmax:
            verbose:
            gg: index to extract from diagonal. 
              Must be 0 <= gg <= nBins
              Default: 0 (for Kappa)
        remake kk covar and fisher with noise
        (copy some code from "Rebuild covar with noise")
        Originally written for kk, but modified to do any gg; ZK, 2018.05.10
    """

    """
    Clkk = Fobj.crossCls[0,0]
    Nlkk = Fobj.noiseCls[0,0]
    dClkk = Fobj.dClVecs[0]
    nCosParams = Fobj.nCosParams
    ells = Fobj.ells

    # add noise to crossCls for making covar
    Clkk2 = Clkk+Nlkk

    print 'Clkk shape: ',Clkk2.shape
    print 'dClkk shape: ',dClkk.shape
    cov_kk2 = 2*Clkk2**2 / (2*ells+1)
    """
    # check gg
    if gg < 0 or gg > Fobj.nBins:
        print 'covfefe!'
        return 0
    
    if gg == 0:
        nParams = Fobj.nCosParams
    else:
        nParams = Fobj.nParams
    cov_kk2 = Fobj.covar[gg,gg]
    dClkk = Fobj.dClVecs[gg]
    Fij_kk2 = np.zeros((nParams,nParams))
    lmin = Fobj.lmin
    minEllInd = myLmin-lmin  # eg. 30-2
    maxEllInd = lmax-lmin 
    for pNum1 in range(nParams):
        for pNum2 in range(pNum1,nParams):
            # the next line does the summation over ell
            Fij_kk2[pNum1,pNum2] = np.sum( dClkk[pNum1,minEllInd:maxEllInd] * \
                                           dClkk[pNum2,minEllInd:maxEllInd] / cov_kk2[minEllInd:maxEllInd])
            Fij_kk2[pNum2,pNum1] = Fij_kk2[pNum1,pNum2]
    
    if verbose:
        print 'parameters: ',Fobj.paramList[:nParams]
        print 'kappa-only, noise-included Fisher matrix (from ell = {0} to {1}):'.format(myLmin,lmax)
        print Fij_kk2
    return Fij_kk2

In [28]:
# get kappa-only Fij.s
Fij_kk2 = getFijkk(Fobj2,myLmin=myLmin,lmax=2000,verbose=False)
#Fij_kk4 = getFijkk(Fobj4,myLmin=myLmin,lmax=2000,verbose=False)
print 'done.'

done.


In [29]:
# get galaxy only Fij.s
Fij_gg2 = np.zeros(Fobj2.Fij.shape)
for binNum in range(1,Fobj2.nBins+1):
    Fij_gg2 += getFijkk(Fobj2,myLmin=myLmin,lmax=2000,verbose=False,gg=binNum)
#Fij_gg4 = np.zeros(Fobj4.Fij.shape)
#for binNum in range(1,Fobj4.nBins+1):
#    Fij_gg4 += getFijkk(Fobj4,myLmin=myLmin,lmax=2000,verbose=False,gg=binNum)
print 'done.'

done.


In [30]:
print Fij_gg2.shape

(25, 25)


# build kTE, gTE, and kgTE Fisher matrices

In [31]:
# get the TE matrices

FijTE2 = Fobj2.makeFisher(myLmin,TE=True)
#FijTE4 = Fobj4.makeFisher(myLmin,TE=True)
print 'done'

done


In [32]:
# create combined Fisher matrices
Fij_kTE2 = Fij_kk2.copy()
Fij_kTE2[:nCosParams,:nCosParams] += FijTE2
#Fij_kTE4 = Fij_kk4.copy()
#Fij_kTE4[:nCosParams,:nCosParams] += FijTE4

Fij_kgTE2_00 = Fij2_00.copy()  # k-unlimited
Fij_kgTE2_00[:nCosParams,:nCosParams] += FijTE2
Fij_kgTE2_01 = Fij2_01.copy()  # kmax1
Fij_kgTE2_01[:nCosParams,:nCosParams] += FijTE2
Fij_kgTE2_02 = Fij2_02.copy()  # kmax2
Fij_kgTE2_02[:nCosParams,:nCosParams] += FijTE2

#Fij_kgTE4_00 = Fij4_00.copy()  # k-unlimited
#Fij_kgTE4_00[:nCosParams,:nCosParams] += FijTE4
#Fij_kgTE4_01 = Fij4_01.copy()  # kmax1
#Fij_kgTE4_01[:nCosParams,:nCosParams] += FijTE4
#Fij_kgTE4_02 = Fij4_02.copy()  # kmax2
#Fij_kgTE4_02[:nCosParams,:nCosParams] += FijTE4

print 'done'

done


In [33]:
# addendum: do gTE as well
#Fij_gTE2_00 = Fij2_00g.copy()  # k-unlimited
#Fij_gTE2_00[:nCosParams,:nCosParams] += FijTE2
Fij_gTE2_01 = Fij2_01g.copy()  # kmax1
Fij_gTE2_01[:nCosParams,:nCosParams] += FijTE2
Fij_gTE2_02 = Fij2_02g.copy()  # kmax2
Fij_gTE2_02[:nCosParams,:nCosParams] += FijTE2

##Fij_gTE4_00 = Fij4_00g.copy()  # k-unlimited
##Fij_gTE4_00[:nCosParams,:nCosParams] += FijTE4
#Fij_gTE4_01 = Fij4_01g.copy()  # kmax1
#Fij_gTE4_01[:nCosParams,:nCosParams] += FijTE4
#Fij_gTE4_02 = Fij4_02g.copy()  # kmax2
#Fij_gTE4_02[:nCosParams,:nCosParams] += FijTE4

print 'done'

done


# Adjust $A_s$ values

In [34]:
# Convert As to 10^9*As
#"""
AsIndex = 3
mult = 1e-9
def rowColMult(array,index,mult):
    array[index,:] *= mult
    array[:,index] *= mult
    return array
    
Fij_kk2      = rowColMult(Fij_kk2,AsIndex,mult)
FijTE2       = rowColMult(FijTE2,AsIndex,mult)
Fij_kTE2     = rowColMult(Fij_kTE2,AsIndex,mult)
Fij2_00      = rowColMult(Fij2_00,AsIndex,mult)
Fij2_01      = rowColMult(Fij2_01,AsIndex,mult)
Fij2_02      = rowColMult(Fij2_02,AsIndex,mult)
Fij_kgTE2_00 = rowColMult(Fij_kgTE2_00,AsIndex,mult)
Fij_kgTE2_01 = rowColMult(Fij_kgTE2_01,AsIndex,mult)
Fij_kgTE2_02 = rowColMult(Fij_kgTE2_02,AsIndex,mult)

#Fij_kk4      = rowColMult(Fij_kk4,AsIndex,mult)
#FijTE4       = rowColMult(FijTE4,AsIndex,mult)
#Fij_kTE4     = rowColMult(Fij_kTE4,AsIndex,mult)
#Fij4_00      = rowColMult(Fij4_00,AsIndex,mult)
#Fij4_01      = rowColMult(Fij4_01,AsIndex,mult)
#Fij4_02      = rowColMult(Fij4_02,AsIndex,mult)
#Fij_kgTE4_00 = rowColMult(Fij_kgTE4_00,AsIndex,mult)
#Fij_kgTE4_01 = rowColMult(Fij_kgTE4_01,AsIndex,mult)
#Fij_kgTE4_02 = rowColMult(Fij_kgTE4_02,AsIndex,mult)


# adjust name
paramList[AsIndex]     = '10^9 As'
paramList2_01[AsIndex] = '10^9 As'
paramList2_02[AsIndex] = '10^9 As'
#paramList4_01[AsIndex] = '10^9 As'
#paramList4_02[AsIndex] = '10^9 As'
print paramList
#"""

['ombh2', 'omch2', 'cosmomc_theta', '10^9 As', 'ns', 'tau', 'mnu', 'w', 'wa', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6', 'bin7', 'bin8', 'bin9', 'bin10', 'bin11', 'bin12', 'bin13', 'bin14', 'bin15', 'bin16']


In [35]:
# the gTE only part

Fij_gTE2_01 = rowColMult(Fij_gTE2_01,AsIndex,mult)
Fij_gTE2_02 = rowColMult(Fij_gTE2_02,AsIndex,mult)
#Fij_gTE4_01 = rowColMult(Fij_gTE4_01,AsIndex,mult)
#Fij_gTE4_02 = rowColMult(Fij_gTE4_02,AsIndex,mult)

# Get $a_p$, $\sigma(w_p)$, $\sigma(w_a)$, FoM

In [36]:
# define a few functions for calculating FoM

# define function for getting ap (pivot a) for each matrix
def getAp(FijInv,iw0=7,iwa=8):
    dw0dwa = FijInv[iw0,iwa]
    dwadwa = FijInv[iwa,iwa]
    return 1+dw0dwa/dwadwa

# define function for getting sigma^2(w_p) (variance of w_p)
def getVarWp(FijInv,iw0=7,iwa=8):
    dw0dw0 = FijInv[iw0,iw0]
    dw0dwa = FijInv[iw0,iwa]
    dwadwa = FijInv[iwa,iwa]
    a_p = getAp(FijInv,iw0,iwa)
    return dw0dw0 +2*(1-a_p)*dw0dwa +(1-a_p)**2*dwadwa

# define function for getting value proportional to FoM
def FoMprop(FijInv,iw0=7,iwa=8):
    sigmaWp = np.sqrt(getVarWp(FijInv,iw0,iwa))
    sigmaWa = np.sqrt(FijInv[iwa,iwa])
    return 1/(sigmaWp*sigmaWa)


In [37]:
# get inverses
FInv_kTE2 = np.linalg.inv(Fij_kTE2)
#FInv_kTE4 = np.linalg.inv(Fij_kTE4)
FInv_kgTE2_01 = np.linalg.inv(Fij_kgTE2_01)
FInv_kgTE2_02 = np.linalg.inv(Fij_kgTE2_02)
#FInv_kgTE4_01 = np.linalg.inv(Fij_kgTE4_01)
#FInv_kgTE4_02 = np.linalg.inv(Fij_kgTE4_02)
FInv_gTE2_01 = np.linalg.inv(Fij_gTE2_01)
FInv_gTE2_02 = np.linalg.inv(Fij_gTE2_02)
#FInv_gTE4_01 = np.linalg.inv(Fij_gTE4_01)
#FInv_gTE4_02 = np.linalg.inv(Fij_gTE4_02)

print 'done.'

done.


In [38]:
# get a_p
ap_kTE2 = getAp(FInv_kTE2)
#ap_kTE4 = getAp(FInv_kTE4)
ap_kgTE2_01 = getAp(FInv_kgTE2_01)
ap_kgTE2_02 = getAp(FInv_kgTE2_02)
#ap_kgTE4_01 = getAp(FInv_kgTE4_01)
#ap_kgTE4_02 = getAp(FInv_kgTE4_02)
ap_gTE2_01 = getAp(FInv_gTE2_01)
ap_gTE2_02 = getAp(FInv_gTE2_02)
#ap_gTE4_01 = getAp(FInv_gTE4_01)
#ap_gTE4_02 = getAp(FInv_gTE4_02)

print 'done.'

done.


In [39]:
# get sigma(wp)
sigWp_kTE2 = np.sqrt(getVarWp(FInv_kTE2))
#sigWp_kTE4 = np.sqrt(getVarWp(FInv_kTE4))
sigWp_kgTE2_01 = np.sqrt(getVarWp(FInv_kgTE2_01))
sigWp_kgTE2_02 = np.sqrt(getVarWp(FInv_kgTE2_02))
#sigWp_kgTE4_01 = np.sqrt(getVarWp(FInv_kgTE4_01))
#sigWp_kgTE4_02 = np.sqrt(getVarWp(FInv_kgTE4_02))
sigWp_gTE2_01 = np.sqrt(getVarWp(FInv_gTE2_01))
sigWp_gTE2_02 = np.sqrt(getVarWp(FInv_gTE2_02))
#sigWp_gTE4_01 = np.sqrt(getVarWp(FInv_gTE4_01))
#sigWp_gTE4_02 = np.sqrt(getVarWp(FInv_gTE4_02))

print 'done.'

done.


In [40]:
# get FoMs
FoM_kTE2 = FoMprop(FInv_kTE2)
#FoM_kTE4 = FoMprop(FInv_kTE4)
FoM_kgTE2_01 = FoMprop(FInv_kgTE2_01)
FoM_kgTE2_02 = FoMprop(FInv_kgTE2_02)
#FoM_kgTE4_01 = FoMprop(FInv_kgTE4_01)
#FoM_kgTE4_02 = FoMprop(FInv_kgTE4_02)
FoM_gTE2_01 = FoMprop(FInv_gTE2_01)
FoM_gTE2_02 = FoMprop(FInv_gTE2_02)
#FoM_gTE4_01 = FoMprop(FInv_gTE4_01)
#FoM_gTE4_02 = FoMprop(FInv_gTE4_02)

print 'done.'

done.


# Show the sigmas

In [41]:
# check sigmas
def getSigmas(Fij):
    #print np.diag(np.linalg.inv(Fij))
    return np.sqrt(np.diag(np.linalg.inv(Fij)))
 
sigmas_kk2 = getSigmas(Fij_kk2)
#sigmas_kk4 = getSigmas(Fij_kk4)
print 'kk ok'

sigmas_TE2 = getSigmas(FijTE2)
#sigmas_TE4 = getSigmas(FijTE4)
print 'TE ok'
    
sigmas_kTE2 = getSigmas(Fij_kTE2)
#sigmas_kTE4 = getSigmas(Fij_kTE4)
print 'kTE ok'

sigmas_kg2_00 = getSigmas(Fij2_00)
sigmas_kg2_01 = getSigmas(Fij2_01)
sigmas_kg2_02 = getSigmas(Fij2_02)
#sigmas_kg4_00 = getSigmas(Fij4_00)
#sigmas_kg4_01 = getSigmas(Fij4_01)
#sigmas_kg4_02 = getSigmas(Fij4_02)
print 'kg ok'

sigmas_kgTE2_00 = getSigmas(Fij_kgTE2_00)
sigmas_kgTE2_01 = getSigmas(Fij_kgTE2_01)
sigmas_kgTE2_02 = getSigmas(Fij_kgTE2_02)
#sigmas_kgTE4_00 = getSigmas(Fij_kgTE4_00)
#sigmas_kgTE4_01 = getSigmas(Fij_kgTE4_01)
#sigmas_kgTE4_02 = getSigmas(Fij_kgTE4_02)
print 'kgTE ok'

# gTE part - Fijs seem to have all zeros for galaxy bin biases.  hm.
#sigmas_gTE2_00 = getSigmas(Fij_gTE2_00)
sigmas_gTE2_01 = getSigmas(Fij_gTE2_01)
sigmas_gTE2_02 = getSigmas(Fij_gTE2_02)
##sigmas_gTE4_00 = getSigmas(Fij_gTE4_00)
#sigmas_gTE4_01 = getSigmas(Fij_gTE4_01)
#sigmas_gTE4_02 = getSigmas(Fij_gTE4_02)

#sigmas_gTE2_00 = getSigmas(Fij_gTE2_00[:nCosParams,:nCosParams])
#sigmas_gTE2_01 = getSigmas(Fij_gTE2_01[:nCosParams,:nCosParams])
#sigmas_gTE2_02 = getSigmas(Fij_gTE2_02[:nCosParams,:nCosParams])
#sigmas_gTE4_00 = getSigmas(Fij_gTE4_00[:nCosParams,:nCosParams])
#sigmas_gTE4_01 = getSigmas(Fij_gTE4_01[:nCosParams,:nCosParams])
#sigmas_gTE4_02 = getSigmas(Fij_gTE4_02[:nCosParams,:nCosParams])
print 'gTE ok'

kk ok
TE ok
kTE ok
kg ok
kgTE ok
gTE ok


In [42]:
# print the table in the ordering that I want for my paper or presentations
from prettytable import PrettyTable

print 'fsky = 1'
print '{0} P(k)'.format(label2)

# mnu index
Imnu = 6
Iwa = 8

# zeros should already have been added to kTE
#print 'g-TE and kg-TE; k_max = {0}'.format(kmax2)
t = PrettyTable(['Parameter','kTE','gTE_01','gTE_02','kgTE_01','kgTE_02'])
    
t.add_row(['sig_mnu',"%5.3g" % sigmas_kTE2[Imnu],"%5.3g" % sigmas_gTE2_01[Imnu],"%5.3g" % sigmas_gTE2_02[Imnu],
           "%5.3g" % sigmas_kgTE2_01[Imnu],"%5.3g" % sigmas_kgTE2_02[Imnu]])
t.add_row(['sig_wa',"%5.3g" % sigmas_kTE2[Iwa],"%5.3g" % sigmas_gTE2_01[Iwa],"%5.3g" % sigmas_gTE2_02[Iwa],
           "%5.3g" % sigmas_kgTE2_01[Iwa],"%5.3g" % sigmas_kgTE2_02[Iwa]])
t.add_row(['sig_wp',"%5.3g" % sigWp_kTE2,"%5.3g" % sigWp_gTE2_01,"%5.3g" % sigWp_gTE2_02,
           "%5.3g" % sigWp_kgTE2_01,"%5.3g" % sigWp_kgTE2_02])
t.add_row(['FoM',"%5.3g" % FoM_kTE2,"%5.3g" % FoM_gTE2_01,"%5.3g" % FoM_gTE2_02,
           "%5.3g" % FoM_kgTE2_01,"%5.3g" % FoM_kgTE2_02])
t.add_row(['a_p',"%5.3g" % ap_kTE2,"%5.3g" % ap_gTE2_01,"%5.3g" % ap_gTE2_02,
           "%5.3g" % ap_kgTE2_01,"%5.3g" % ap_kgTE2_02])

print t

fsky = 1
linear P(k)
+-----------+-------+--------+--------+---------+---------+
| Parameter |  kTE  | gTE_01 | gTE_02 | kgTE_01 | kgTE_02 |
+-----------+-------+--------+--------+---------+---------+
|  sig_mnu  | 0.174 | 0.0329 | 0.0291 |  0.0288 |  0.0229 |
|   sig_wa  |  6.94 |  1.02  | 0.572  |  0.419  |  0.306  |
|   sig_wp  | 0.123 | 0.064  | 0.0423 |  0.0321 |   0.02  |
|    FoM    |  1.17 |  15.4  |  41.3  |   74.4  |    164  |
|    a_p    | 0.523 |  0.53  | 0.555  |  0.547  |  0.552  |
+-----------+-------+--------+--------+---------+---------+


In [43]:
# now with fsky
fsky = 0.44
f12 = fsky**-0.5

print 'f_sky = ',fsky
print '{0} P(k)'.format(label2)

# zeros should already have been added to kTE
#print 'g-TE and kg-TE; k_max = {0}'.format(kmax2)
t = PrettyTable(['Parameter','kTE','gTE_01','gTE_02','kgTE_01','kgTE_02'])
    
t.add_row(['sig_mnu',"%5.3g" % (sigmas_kTE2[Imnu]*f12),"%5.3g" % (sigmas_gTE2_01[Imnu]*f12),"%5.3g" % (sigmas_gTE2_02[Imnu]*f12),
           "%5.3g" % (sigmas_kgTE2_01[Imnu]*f12),"%5.3g" % (sigmas_kgTE2_02[Imnu]*f12)])
t.add_row(['sig_wa',"%5.3g" % (sigmas_kTE2[Iwa]*f12),"%5.3g" % (sigmas_gTE2_01[Iwa]*f12),"%5.3g" % (sigmas_gTE2_02[Iwa]*f12),
           "%5.3g" % (sigmas_kgTE2_01[Iwa]*f12),"%5.3g" % (sigmas_kgTE2_02[Iwa]*f12)])
t.add_row(['sig_wp',"%5.3g" % (sigWp_kTE2*f12),"%5.3g" % (sigWp_gTE2_01*f12),"%5.3g" % (sigWp_gTE2_02*f12),
           "%5.3g" % (sigWp_kgTE2_01*f12),"%5.3g" % (sigWp_kgTE2_02*f12)])
t.add_row(['FoM',"%5.3g" % (FoM_kTE2*fsky),"%5.3g" % (FoM_gTE2_01*fsky),"%5.3g" % (FoM_gTE2_02*fsky),
           "%5.3g" % (FoM_kgTE2_01*fsky),"%5.3g" % (FoM_kgTE2_02*fsky)])
t.add_row(['a_p',"%5.3g" % ap_kTE2,"%5.3g" % ap_gTE2_01,"%5.3g" % ap_gTE2_02,
           "%5.3g" % ap_kgTE2_01,"%5.3g" % ap_kgTE2_02])

print t

#print '{0} P(k)'.format(label4)
#
## zeros should already have been added to kTE
##print 'g-TE and kg-TE; k_max = {0}'.format(kmax2)
#t = PrettyTable(['Parameter','kTE','gTE_01','gTE_02','kgTE_01','kgTE_02'])
#    
#t.add_row(['sig_mnu',"%5.3g" % (sigmas_kTE4[Imnu]*f12),"%5.3g" % (sigmas_gTE4_01[Imnu]*f12),"%5.3g" % (sigmas_gTE4_02[Imnu]*f12),
#           "%5.3g" % (sigmas_kgTE4_01[Imnu]*f12),"%5.3g" % (sigmas_kgTE4_02[Imnu]*f12)])
#t.add_row(['sig_wa',"%5.3g" % (sigmas_kTE4[Iwa]*f12),"%5.3g" % (sigmas_gTE4_01[Iwa]*f12),"%5.3g" % (sigmas_gTE4_02[Iwa]*f12),
#           "%5.3g" % (sigmas_kgTE4_01[Iwa]*f12),"%5.3g" % (sigmas_kgTE4_02[Iwa]*f12)])
#t.add_row(['sig_wp',"%5.3g" % (sigWp_kTE4*f12),"%5.3g" % (sigWp_gTE4_01*f12),"%5.3g" % (sigWp_gTE4_02*f12),
#           "%5.3g" % (sigWp_kgTE4_01*f12),"%5.3g" % (sigWp_kgTE4_02*f12)])
#t.add_row(['FoM',"%5.3g" % (FoM_kTE4*fsky),"%5.3g" % (FoM_gTE4_01*fsky),"%5.3g" % (FoM_gTE4_02*fsky),
#           "%5.3g" % (FoM_kgTE4_01*fsky),"%5.3g" % (FoM_kgTE4_02*fsky)])
#t.add_row(['a_p',"%5.3g" % ap_kTE4,"%5.3g" % ap_gTE4_01,"%5.3g" % ap_gTE4_02,
#           "%5.3g" % ap_kgTE4_01,"%5.3g" % ap_kgTE4_02])
#
#print t

f_sky =  0.44
linear P(k)
+-----------+-------+--------+--------+---------+---------+
| Parameter |  kTE  | gTE_01 | gTE_02 | kgTE_01 | kgTE_02 |
+-----------+-------+--------+--------+---------+---------+
|  sig_mnu  | 0.262 | 0.0496 | 0.0438 |  0.0435 |  0.0345 |
|   sig_wa  |  10.5 |  1.53  | 0.862  |  0.632  |  0.461  |
|   sig_wp  | 0.185 | 0.0965 | 0.0638 |  0.0484 |  0.0301 |
|    FoM    | 0.516 |  6.75  |  18.2  |   32.7  |   72.1  |
|    a_p    | 0.523 |  0.53  | 0.555  |  0.547  |  0.552  |
+-----------+-------+--------+--------+---------+---------+


# Get fixed-w versions

In [44]:
print Fobj2.paramList

['ombh2', 'omch2', 'cosmomc_theta', '10^9 As', 'ns', 'tau', 'mnu', 'w', 'wa', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6', 'bin7', 'bin8', 'bin9', 'bin10', 'bin11', 'bin12', 'bin13', 'bin14', 'bin15', 'bin16']


In [45]:
fixW = False
if fixW:
    # fix w0, wa
    w0Index = 7
    waIndex = 8

    Fij_kk2_now = fixParam(Fij_kk2,waIndex)
    Fij_kk2_now = fixParam(Fij_kk2_now,w0Index)

    Fij_TE2_now = fixParam(FijTE2,waIndex)
    Fij_TE2_now = fixParam(Fij_TE2_now,w0Index)

    Fij_kTE2_now = fixParam(Fij_kTE2,waIndex)
    Fij_kTE2_now = fixParam(Fij_kTE2_now,w0Index)

    Fij_kg2_00_now = fixParam(Fij2_00,waIndex)
    Fij_kg2_00_now = fixParam(Fij_kg2_00_now,w0Index)
    Fij_kg2_01_now = fixParam(Fij2_01,waIndex)
    Fij_kg2_01_now = fixParam(Fij_kg2_01_now,w0Index)
    Fij_kg2_02_now = fixParam(Fij2_02,waIndex)
    Fij_kg2_02_now = fixParam(Fij_kg2_02_now,w0Index)

    Fij_kgTE2_00_now = fixParam(Fij_kgTE2_00,waIndex)
    Fij_kgTE2_00_now = fixParam(Fij_kgTE2_00_now,w0Index)
    Fij_kgTE2_01_now = fixParam(Fij_kgTE2_01,waIndex)
    Fij_kgTE2_01_now = fixParam(Fij_kgTE2_01_now,w0Index)
    Fij_kgTE2_02_now = fixParam(Fij_kgTE2_02,waIndex)
    Fij_kgTE2_02_now = fixParam(Fij_kgTE2_02_now,w0Index)

    #Fij_gTE2_00_now = fixParam(Fij_gTE2_00,waIndex)
    #Fij_gTE2_00_now = fixParam(Fij_gTE2_00_now,w0Index)
    Fij_gTE2_01_now = fixParam(Fij_gTE2_01,waIndex)
    Fij_gTE2_01_now = fixParam(Fij_gTE2_01_now,w0Index)
    Fij_gTE2_02_now = fixParam(Fij_gTE2_02,waIndex)
    Fij_gTE2_02_now = fixParam(Fij_gTE2_02_now,w0Index)




    # get sigmas
    sigmas_kk2_now = getSigmas(Fij_kk2_now)
    print 'kk ok'

    sigmas_TE2_now = getSigmas(Fij_TE2_now)
    print 'TE ok'

    sigmas_kTE2_now = getSigmas(Fij_kTE2_now)
    print 'kTE ok'

    sigmas_kg2_00_now = getSigmas(Fij_kg2_00_now)
    sigmas_kg2_01_now = getSigmas(Fij_kg2_01_now)
    sigmas_kg2_02_now = getSigmas(Fij_kg2_02_now)
    print 'kg ok'

    sigmas_kgTE2_00_now = getSigmas(Fij_kgTE2_00_now)
    sigmas_kgTE2_01_now = getSigmas(Fij_kgTE2_01_now)
    sigmas_kgTE2_02_now = getSigmas(Fij_kgTE2_02_now)
    print 'kgTE ok'

    # gTE part - Fijs seem to have all zeros for galaxy bin biases.  hm.
    #sigmas_gTE2_00_now = getSigmas(Fij_gTE2_00_now)
    sigmas_gTE2_01_now = getSigmas(Fij_gTE2_01_now)
    sigmas_gTE2_02_now = getSigmas(Fij_gTE2_02_now)
    print 'gTE ok'

In [46]:
if fixW:
    from prettytable import PrettyTable 

    print 'kappa-TE and kg-TE; w-fixed; biases marg; no k-limit'
    t = PrettyTable(['Parameter','sigmas_kTE2','sigmas_kgTE2_00','percent diff.'])
    for i,p in enumerate(paramList2_01[:7]):
        Zi = sigmas_kTE2_now[i]
        Bi = sigmas_kgTE2_00_now[i]
        diff = (Bi-Zi)/Zi *100
        t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
    print t

    print 'kappa-TE and kg-TE; w-fixed; biases marg; k_max = {0}'.format(kmax2)
    #t = PrettyTable(['Parameter','sigmas_kTE2','sigmas_kgTE2_02','percent diff.'])
    t = PrettyTable(['Parameter','sigmas_kTE','sigmas_kgTE','percent diff.'])
    for i,p in enumerate(paramList2_02[:7]):
        Zi = sigmas_kTE2_now[i]
        Bi = sigmas_kgTE2_02_now[i]
        diff = (Bi-Zi)/Zi *100
        t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
    print t

    print 'kappa-TE and kg-TE; w-fixed; biases marg; k_max = {0}'.format(kmax1)
    t = PrettyTable(['Parameter','sigmas_kTE2','sigmas_kgTE2_01','percent diff.'])
    for i,p in enumerate(paramList2_01[:7]):
        Zi = sigmas_kTE2_now[i]
        Bi = sigmas_kgTE2_01_now[i]
        diff = (Bi-Zi)/Zi *100
        t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
    print t

    print 'switch to gTE'

    print 'g-TE and kg-TE; w-fixed; biases marg; k_max = {0}'.format(kmax1)
    #t = PrettyTable(['Parameter','sigmas_gTE2_02','sigmas_kgTE2_02','percent diff.'])
    t = PrettyTable(['Parameter','sigmas_gTE','sigmas_kgTE','percent diff.'])
    for i,p in enumerate(paramList2_01[:7]):
        Zi = sigmas_gTE2_01_now[i]
        Bi = sigmas_kgTE2_01_now[i]
        diff = (Bi-Zi)/Zi *100
        t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
    print t

    print 'g-TE and kg-TE; w-fixed; biases marg; k_max = {0}'.format(kmax2)
    #t = PrettyTable(['Parameter','sigmas_gTE2_02','sigmas_kgTE2_02','percent diff.'])
    t = PrettyTable(['Parameter','sigmas_gTE','sigmas_kgTE','percent diff.'])
    for i,p in enumerate(paramList2_02[:7]):
        Zi = sigmas_gTE2_02_now[i]
        Bi = sigmas_kgTE2_02_now[i]
        diff = (Bi-Zi)/Zi *100
        t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
    print t

    print 'include TE'
    print 'TE, kappa-TE, and g-TE; w-fixed; biases marg; k_max = {0} or {1}'.format(kmax1,kmax2)
    #t = PrettyTable(['Parameter','sigmas_kTE2','sigmas_kgTE2_02','percent diff.'])
    t = PrettyTable(['Parameter','sigmas_TE','sigmas_kTE','sigmas_gTE_1','sigmas_gTE_2'])
    for i,p in enumerate(paramList2_02[:7]):
        TE = sigmas_TE2_now[i]
        kTE = sigmas_kTE2_now[i]
        gTE1 = sigmas_gTE2_01_now[i]
        gTE2 = sigmas_gTE2_02_now[i]
        t.add_row([p,"%5.3g" % TE,"%5.3g" % kTE, "%5.3g" % gTE1, "%5.3g" % gTE2 ] )
    print t

# kk, TE, kTE, kg

In [47]:
from prettytable import PrettyTable

print 'fsky = 1'
print '{0} P(k)'.format(label2)

print 'kappa-only and kappa-TE'
t = PrettyTable(['Parameter','sigmas_kk2','sigmas_kTE2','percent diff.'])
for i,p in enumerate(paramList[:nCosParams]):
    Zi = sigmas_kk2[i]
    Bi = sigmas_kTE2[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'TE-only and kappa-TE'
t = PrettyTable(['Parameter','sigmas_TE2','sigmas_kTE2','percent diff.'])
for i,p in enumerate(paramList[:nCosParams]):
    Zi = sigmas_TE2[i]
    Bi = sigmas_kTE2[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

# add extra zeros to kk
sigmas_kk2 = np.append(sigmas_kk2,np.zeros(nBins))

print 'kappa-only and kappa-galaxies, no k-limit'
t = PrettyTable(['Parameter','sigmas_kk2','sigmas_kg2_00','percent diff.'])
for i,p in enumerate(paramList):
    Zi = sigmas_kk2[i]
    Bi = sigmas_kg2_00[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kappa-only and kappa-galaxies, k_max = {0}'.format(kmax2)
t = PrettyTable(['Parameter','sigmas_kk2','sigmas_kg2_02','percent diff.'])
for i,p in enumerate(paramList2_02):
    Zi = sigmas_kk2[i]
    Bi = sigmas_kg2_02[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kappa-only and kappa-galaxies, k_max = {0}'.format(kmax1)
t = PrettyTable(['Parameter','sigmas_kk2','sigmas_kg2_01','percent diff.'])
for i,p in enumerate(paramList2_01):
    Zi = sigmas_kk2[i]
    Bi = sigmas_kg2_01[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t


fsky = 1
linear P(k)
kappa-only and kappa-TE
+---------------+------------+-------------+---------------+
|   Parameter   | sigmas_kk2 | sigmas_kTE2 | percent diff. |
+---------------+------------+-------------+---------------+
|     ombh2     |   0.0837   |   8.75e-06  |      -100     |
|     omch2     |   0.211    |   0.000392  |     -99.8     |
| cosmomc_theta |  0.00201   |   1.49e-06  |     -99.9     |
|    10^9 As    |    5.76    |    0.127    |     -97.8     |
|       ns      |   0.196    |   0.00134   |     -99.3     |
|      tau      |     577    |     0.03    |      -100     |
|      mnu      |    2.35    |    0.174    |     -92.6     |
|       w       |    19.9    |     3.32    |     -83.3     |
|       wa      |      39    |     6.94    |     -82.2     |
+---------------+------------+-------------+---------------+
TE-only and kappa-TE
+---------------+------------+-------------+---------------+
|   Parameter   | sigmas_TE2 | sigmas_kTE2 | percent diff. |
+---------------+--

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in double_scalars
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in double_scalars
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in double_scalars


# kTE, kgTE

In [48]:
# gTE should be in this section, but I didn't add it yet.

In [49]:
from prettytable import PrettyTable

print 'fsky = 1'
print '{0} P(k)'.format(label2)

# add extra zeros to kTE
sigmas_kTE2 = np.append(sigmas_kTE2,np.zeros(nBins))

print 'kappa-TE and kg-TE; no k-limit'
t = PrettyTable(['Parameter','sigmas_kTE2','sigmas_kgTE2_00','percent diff.'])
for i,p in enumerate(paramList2_01):
    Zi = sigmas_kTE2[i]
    Bi = sigmas_kgTE2_00[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kappa-TE and kg-TE; k_max = {0}'.format(kmax2)
t = PrettyTable(['Parameter','sigmas_kTE2','sigmas_kgTE2_02','percent diff.'])
for i,p in enumerate(paramList2_02):
    Zi = sigmas_kTE2[i]
    Bi = sigmas_kgTE2_02[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kappa-TE and kg-TE; k_max = {0}'.format(kmax1)
t = PrettyTable(['Parameter','sigmas_kTE2','sigmas_kgTE2_01','percent diff.'])
for i,p in enumerate(paramList2_01):
    Zi = sigmas_kTE2[i]
    Bi = sigmas_kgTE2_01[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t



fsky = 1
linear P(k)
kappa-TE and kg-TE; no k-limit
+---------------+-------------+-----------------+---------------+
|   Parameter   | sigmas_kTE2 | sigmas_kgTE2_00 | percent diff. |
+---------------+-------------+-----------------+---------------+
|     ombh2     |   8.75e-06  |     7.18e-06    |       -18     |
|     omch2     |   0.000392  |     0.000193    |     -50.7     |
| cosmomc_theta |   1.49e-06  |     2.65e-07    |     -82.2     |
|    10^9 As    |    0.127    |      0.0153     |     -87.9     |
|       ns      |   0.00134   |     0.000802    |     -40.1     |
|      tau      |     0.03    |     0.00386     |     -87.1     |
|      mnu      |    0.174    |      0.0171     |     -90.2     |
|       w       |     3.32    |      0.0617     |     -98.1     |
|       wa      |     6.94    |      0.151      |     -97.8     |
|      bin2     |        0    |     0.00283     |       inf     |
|      bin3     |        0    |     0.00279     |       inf     |
|      bin4     |       

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in double_scalars
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in double_scalars


# gTE, kgTE

In [50]:
from prettytable import PrettyTable

print 'fsky = 1'
print '{0} P(k)'.format(label2)

# add extra zeros to kTE
sigmas_kTE2 = np.append(sigmas_kTE2,np.zeros(nBins))

#print 'g-TE and kg-TE; no k-limit'
#t = PrettyTable(['Parameter','sigmas_gTE2','sigmas_kgTE2_00','percent diff.'])
#for i,p in enumerate(paramList2_01):
#    Zi = sigmas_gTE2[i]
#    Bi = sigmas_kgTE2_00[i]
#    diff = (Bi-Zi)/Zi *100
#    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
#print t

print 'g-TE and kg-TE; k_max = {0}'.format(kmax2)
t = PrettyTable(['Parameter','sigmas_gTE2_02','sigmas_kgTE2_02','percent diff.'])
for i,p in enumerate(paramList2_02):
    Zi = sigmas_gTE2_02[i]
    Bi = sigmas_kgTE2_02[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'g-TE and kg-TE; k_max = {0}'.format(kmax1)
t = PrettyTable(['Parameter','sigmas_gTE2_01','sigmas_kgTE2_01','percent diff.'])
for i,p in enumerate(paramList2_01):
    Zi = sigmas_gTE2_01[i]
    Bi = sigmas_kgTE2_01[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t



fsky = 1
linear P(k)
g-TE and kg-TE; k_max = 0.198020407391
+---------------+----------------+-----------------+---------------+
|   Parameter   | sigmas_gTE2_02 | sigmas_kgTE2_02 | percent diff. |
+---------------+----------------+-----------------+---------------+
|     ombh2     |    8.36e-06    |     8.22e-06    |     -1.63     |
|     omch2     |    0.000359    |     0.000245    |     -31.7     |
| cosmomc_theta |    4.27e-07    |     3.22e-07    |     -24.6     |
|    10^9 As    |     0.292      |      0.0177     |     -93.9     |
|       ns      |    0.00109     |     0.000973    |     -10.5     |
|      tau      |     0.0685     |     0.00446     |     -93.5     |
|      mnu      |     0.0291     |      0.0229     |     -21.3     |
|       w       |     0.258      |      0.138      |     -46.4     |
|       wa      |     0.572      |      0.306      |     -46.6     |
|      bin2     |     0.0908     |      0.0105     |     -88.5     |
|      bin3     |     0.103      |     0.00

# TE, kg, kgTE

In [51]:
from prettytable import PrettyTable

print 'fsky = 1'
print '{0} P(k)'.format(label2)

print 'kg and kgTE, no k-limit'
t = PrettyTable(['Parameter','sigmas_kg2_00','sigmas_kgTE2_00','percent diff.'])
for i,p in enumerate(paramList):
    Zi = sigmas_kg2_00[i]
    Bi = sigmas_kgTE2_00[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kg and kgTE, k_max = {0}'.format(kmax2)
t = PrettyTable(['Parameter','sigmas_kg2_02','sigmas_kgTE2_02','percent diff.'])
for i,p in enumerate(paramList2_02):
    Zi = sigmas_kg2_02[i]
    Bi = sigmas_kgTE2_02[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kg and kgTE, k_max = {0}'.format(kmax1)
t = PrettyTable(['Parameter','sigmas_kg2_01','sigmas_kgTE2_01','percent diff.'])
for i,p in enumerate(paramList2_01):
    Zi = sigmas_kg2_01[i]
    Bi = sigmas_kgTE2_01[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

# add extra zeros to kTE
sigmas_TE2 = np.append(sigmas_TE2,np.zeros(nBins))

print 'TE and kgTE, no k-limit'
t = PrettyTable(['Parameter','sigmas_TE2','sigmas_kgTE2_00','percent diff.'])
for i,p in enumerate(paramList):
    Zi = sigmas_TE2[i]
    Bi = sigmas_kgTE2_00[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'TE and kgTE, k_max = {0}'.format(kmax2)
t = PrettyTable(['Parameter','sigmas_TE2','sigmas_kgTE2_02','percent diff.'])
for i,p in enumerate(paramList2_02):
    Zi = sigmas_TE2[i]
    Bi = sigmas_kgTE2_02[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'TE and kgTE, k_max = {0}'.format(kmax1)
t = PrettyTable(['Parameter','sigmas_TE2','sigmas_kgTE2_01','percent diff.'])
for i,p in enumerate(paramList2_01):
    Zi = sigmas_TE2[i]
    Bi = sigmas_kgTE2_01[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t




fsky = 1
linear P(k)
kg and kgTE, no k-limit
+---------------+---------------+-----------------+---------------+
|   Parameter   | sigmas_kg2_00 | sigmas_kgTE2_00 | percent diff. |
+---------------+---------------+-----------------+---------------+
|     ombh2     |    0.000744   |     7.18e-06    |       -99     |
|     omch2     |    0.00209    |     0.000193    |     -90.8     |
| cosmomc_theta |    2.96e-05   |     2.65e-07    |     -99.1     |
|    10^9 As    |     0.0257    |      0.0153     |     -40.3     |
|       ns      |    0.00272    |     0.000802    |     -70.5     |
|      tau      |      5.87     |     0.00386     |     -99.9     |
|      mnu      |     0.035     |      0.0171     |     -51.3     |
|       w       |     0.079     |      0.0617     |     -21.9     |
|       wa      |     0.208     |      0.151      |     -27.5     |
|      bin1     |     0.0055    |     0.00283     |     -48.4     |
|      bin2     |    0.00513    |     0.00279     |     -45.6     |
|  

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in double_scalars
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in double_scalars
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in double_scalars


# print the table in the ordering that I want for my paper or presentations

In [52]:
# print the table in the ordering that I want for my paper or presentations
print 'fsky = 1'
print '{0} P(k)'.format(label2)

# zeros should already have been added to kTE
#print 'g-TE and kg-TE; k_max = {0}'.format(kmax2)
t = PrettyTable(['Parameter','kTE','gTE_01','gTE_02','kgTE_01','kgTE_02'])
for i,p in enumerate(paramList2_02):
    kTE = sigmas_kTE2[i]
    gTE1 = sigmas_gTE2_01[i]
    kgTE1 = sigmas_kgTE2_01[i]
    gTE2 = sigmas_gTE2_02[i]
    kgTE2 = sigmas_kgTE2_02[i]
    t.add_row([p,"%5.3g" % kTE,"%5.3g" % gTE1,"%5.3g" % gTE2, "%5.3g" % kgTE1,"%5.3g" % kgTE2] )
print t

fsky = 1
linear P(k)
+---------------+----------+----------+----------+----------+----------+
|   Parameter   |   kTE    |  gTE_01  |  gTE_02  | kgTE_01  | kgTE_02  |
+---------------+----------+----------+----------+----------+----------+
|     ombh2     | 8.75e-06 | 8.52e-06 | 8.36e-06 | 8.36e-06 | 8.22e-06 |
|     omch2     | 0.000392 | 0.000378 | 0.000359 | 0.000323 | 0.000245 |
| cosmomc_theta | 1.49e-06 | 4.46e-07 | 4.27e-07 | 3.92e-07 | 3.22e-07 |
|    10^9 As    |  0.127   |  0.292   |  0.292   |  0.0241  |  0.0177  |
|       ns      | 0.00134  | 0.00111  | 0.00109  | 0.00103  | 0.000973 |
|      tau      |   0.03   |  0.0687  |  0.0685  | 0.00592  | 0.00446  |
|      mnu      |  0.174   |  0.0329  |  0.0291  |  0.0288  |  0.0229  |
|       w       |   3.32   |  0.483   |  0.258   |  0.192   |  0.138   |
|       wa      |   6.94   |   1.02   |  0.572   |  0.419   |  0.306   |
|      bin2     |      0   |  0.0965  |  0.0908  |  0.0197  |  0.0105  |
|      bin3     |      0   |  

In [53]:
# do kTE, gTE, kgTE again, but with wa fixed this time
w0Index = 7
waIndex = 8

Fij_kk2_nowa = fixParam(Fij_kk2,waIndex)
#Fij_kk4_nowa = fixParam(Fij_kk4,waIndex)

Fij_TE2_nowa = fixParam(FijTE2,waIndex)
#Fij_TE4_nowa = fixParam(FijTE4,waIndex)

Fij_kTE2_nowa = fixParam(Fij_kTE2,waIndex)
#Fij_kTE4_nowa = fixParam(Fij_kTE4,waIndex)

Fij_kg2_00_nowa = fixParam(Fij2_00,waIndex)
Fij_kg2_01_nowa = fixParam(Fij2_01,waIndex)
Fij_kg2_02_nowa = fixParam(Fij2_02,waIndex)
#Fij_kg4_00_nowa = fixParam(Fij4_00,waIndex)
#Fij_kg4_01_nowa = fixParam(Fij4_01,waIndex)
#Fij_kg4_02_nowa = fixParam(Fij4_02,waIndex)

Fij_kgTE2_00_nowa = fixParam(Fij_kgTE2_00,waIndex)
Fij_kgTE2_01_nowa = fixParam(Fij_kgTE2_01,waIndex)
Fij_kgTE2_02_nowa = fixParam(Fij_kgTE2_02,waIndex)
#Fij_kgTE4_00_nowa = fixParam(Fij_kgTE4_00,waIndex)
#Fij_kgTE4_01_nowa = fixParam(Fij_kgTE4_01,waIndex)
#Fij_kgTE4_02_nowa = fixParam(Fij_kgTE4_02,waIndex)

Fij_gTE2_01_nowa = fixParam(Fij_gTE2_01,waIndex)
Fij_gTE2_02_nowa = fixParam(Fij_gTE2_02,waIndex)
#Fij_gTE4_01_nowa = fixParam(Fij_gTE4_01,waIndex)
#Fij_gTE4_02_nowa = fixParam(Fij_gTE4_02,waIndex)


# get sigmas
sigmas_kk2_nowa = getSigmas(Fij_kk2_nowa)
#sigmas_kk4_nowa = getSigmas(Fij_kk4_nowa)
print 'kk ok'

sigmas_TE2_nowa = getSigmas(Fij_TE2_nowa)
#sigmas_TE4_nowa = getSigmas(Fij_TE4_nowa)
print 'TE ok'
    
sigmas_kTE2_nowa = getSigmas(Fij_kTE2_nowa)
#sigmas_kTE4_nowa = getSigmas(Fij_kTE4_nowa)
print 'kTE ok'

sigmas_kg2_00_nowa = getSigmas(Fij_kg2_00_nowa)
sigmas_kg2_01_nowa = getSigmas(Fij_kg2_01_nowa)
sigmas_kg2_02_nowa = getSigmas(Fij_kg2_02_nowa)
#sigmas_kg4_00_nowa = getSigmas(Fij_kg4_00_nowa)
#sigmas_kg4_01_nowa = getSigmas(Fij_kg4_01_nowa)
#sigmas_kg4_02_nowa = getSigmas(Fij_kg4_02_nowa)
print 'kg ok'

sigmas_kgTE2_00_nowa = getSigmas(Fij_kgTE2_00_nowa)
sigmas_kgTE2_01_nowa = getSigmas(Fij_kgTE2_01_nowa)
sigmas_kgTE2_02_nowa = getSigmas(Fij_kgTE2_02_nowa)
#sigmas_kgTE4_00_nowa = getSigmas(Fij_kgTE4_00_nowa)
#sigmas_kgTE4_01_nowa = getSigmas(Fij_kgTE4_01_nowa)
#sigmas_kgTE4_02_nowa = getSigmas(Fij_kgTE4_02_nowa)
print 'kgTE ok'

sigmas_gTE2_01_nowa = getSigmas(Fij_gTE2_01_nowa)
sigmas_gTE2_02_nowa = getSigmas(Fij_gTE2_02_nowa)
#sigmas_gTE4_01_nowa = getSigmas(Fij_gTE4_01_nowa)
#sigmas_gTE4_02_nowa = getSigmas(Fij_gTE4_02_nowa)
print 'gTE ok'

kk ok
TE ok
kTE ok
kg ok
kgTE ok
gTE ok


In [54]:
sigmas_kTE2_nowa = np.append(sigmas_kTE2_nowa,np.zeros(nBins))
#sigmas_kTE4_nowa = np.append(sigmas_kTE4_nowa,np.zeros(nBins))

In [55]:
# print the table in the ordering that I want for my paper or presentations
paramList2_fwa = np.delete(paramList2_02,waIndex)

print 'fsky = 1'
print '{0} P(k)'.format(label2)

print 'now with fixed w_a'
# zeros should already have been added to kTE
#print 'g-TE and kg-TE; k_max = {0}'.format(kmax2)
t = PrettyTable(['Parameter','kTE','gTE_01','gTE_02','kgTE_01','kgTE_02'])
for i,p in enumerate(paramList2_fwa):
    kTE = sigmas_kTE2_nowa[i]
    gTE1 = sigmas_gTE2_01_nowa[i]
    kgTE1 = sigmas_kgTE2_01_nowa[i]
    gTE2 = sigmas_gTE2_02_nowa[i]
    kgTE2 = sigmas_kgTE2_02_nowa[i]
    t.add_row([p,"%5.3g" % kTE,"%5.3g" % gTE1,"%5.3g" % gTE2, "%5.3g" % kgTE1,"%5.3g" % kgTE2] )
print t

fsky = 1
linear P(k)
now with fixed w_a
+---------------+----------+----------+----------+----------+----------+
|   Parameter   |   kTE    |  gTE_01  |  gTE_02  | kgTE_01  | kgTE_02  |
+---------------+----------+----------+----------+----------+----------+
|     ombh2     | 8.57e-06 | 8.52e-06 | 8.35e-06 | 8.36e-06 | 8.22e-06 |
|     omch2     | 0.000392 | 0.000363 | 0.000314 | 0.000289 | 0.000212 |
| cosmomc_theta | 7.96e-07 | 4.23e-07 | 3.71e-07 | 3.55e-07 | 2.83e-07 |
|    10^9 As    |  0.112   |  0.292   |  0.292   |  0.0212  |  0.0161  |
|       ns      | 0.00119  |  0.0011  | 0.00106  |  0.001   | 0.000952 |
|      tau      |  0.0263  |  0.0687  |  0.0685  | 0.00518  | 0.00406  |
|      mnu      |  0.116   |  0.0329  |  0.0269  |  0.0276  |  0.0208  |
|       w       |  0.123   |  0.064   |  0.0423  |  0.0321  |   0.02   |
|      bin2     |      0   |  0.0943  |  0.0906  |  0.016   | 0.00783  |
|      bin3     |      0   |  0.106   |  0.103   |  0.012   | 0.00616  |
|      bin4

# some figure of merit prep

In [56]:
# define a few functions for calculating FoM
# copied to earlier
"""

# define function for getting ap (pivot a) for each matrix
def getAp(FijInv,iw0=8,iwa=7):
    dw0dwa = FijInv[iw0,iwa]
    dwadwa = FijInv[iwa,iwa]
    return 1+dw0dwa/dwadwa

# define function for getting sigma^2(w_p) (variance of w_p)
def getVarWp(FijInv,iw0=8,iwa=7):
    dw0dw0 = FijInv[iw0,iw0]
    dw0dwa = FijInv[iw0,iwa]
    dwadwa = FijInv[iwa,iwa]
    a_p = getAp(FijInv,iw0,iwa)
    return dw0dw0 +2*(1-a_p)*dw0dwa +(1-a_p)**2*dwadwa

# define function for getting value proportional to FoM
def FoMprop(FijInv,iw0=8,iwa=7):
    sigmaWp = np.sqrt(getVarWp(FijInv,iw0,iwa))
    sigmaWa = np.sqrt(FijInv[iwa,iwa])
    return 1/(sigmaWp*sigmaWa)
"""

'\n\n# define function for getting ap (pivot a) for each matrix\ndef getAp(FijInv,iw0=8,iwa=7):\n    dw0dwa = FijInv[iw0,iwa]\n    dwadwa = FijInv[iwa,iwa]\n    return 1+dw0dwa/dwadwa\n\n# define function for getting sigma^2(w_p) (variance of w_p)\ndef getVarWp(FijInv,iw0=8,iwa=7):\n    dw0dw0 = FijInv[iw0,iw0]\n    dw0dwa = FijInv[iw0,iwa]\n    dwadwa = FijInv[iwa,iwa]\n    a_p = getAp(FijInv,iw0,iwa)\n    return dw0dw0 +2*(1-a_p)*dw0dwa +(1-a_p)**2*dwadwa\n\n# define function for getting value proportional to FoM\ndef FoMprop(FijInv,iw0=8,iwa=7):\n    sigmaWp = np.sqrt(getVarWp(FijInv,iw0,iwa))\n    sigmaWa = np.sqrt(FijInv[iwa,iwa])\n    return 1/(sigmaWp*sigmaWa)\n'

# plot $\sigma(w_0), \sigma(w_a)$ as functions of $k_{max}, z_{max}$

In [57]:
# this was executed earlier:
"""
print 'starting nonlinear P(k), kmax = {0}...'.format(kmax1)
covarBins2_01,dClVecsBins2_01,lmaxByBin2_01,preMaskBins2_01 = getKlimited(binEdges,chiOfZ2,Fobj2,kmax=kmax1)
print 'starting nonlinear P(k), kmax = {0}...'.format(kmax2)
covarBins2_02,dClVecsBins2_02,lmaxByBin2_02,preMaskBins2_02 = getKlimited(binEdges,chiOfZ2,Fobj2,kmax=kmax2)

print 'starting linear P(k), kmax = {0}...'.format(kmax1)
covarBins4_01,dClVecsBins4_01,lmaxByBin4_01,preMaskBins4_01 = getKlimited(binEdges,chiOfZ4,Fobj4,kmax=kmax1)
print 'starting linear P(k), kmax = {0}...'.format(kmax2)
covarBins4_02,dClVecsBins4_02,lmaxByBin4_02,preMaskBins4_02 = getKlimited(binEdges,chiOfZ4,Fobj4,kmax=kmax2)
"""

"\nprint 'starting nonlinear P(k), kmax = {0}...'.format(kmax1)\ncovarBins2_01,dClVecsBins2_01,lmaxByBin2_01,preMaskBins2_01 = getKlimited(binEdges,chiOfZ2,Fobj2,kmax=kmax1)\nprint 'starting nonlinear P(k), kmax = {0}...'.format(kmax2)\ncovarBins2_02,dClVecsBins2_02,lmaxByBin2_02,preMaskBins2_02 = getKlimited(binEdges,chiOfZ2,Fobj2,kmax=kmax2)\n\nprint 'starting linear P(k), kmax = {0}...'.format(kmax1)\ncovarBins4_01,dClVecsBins4_01,lmaxByBin4_01,preMaskBins4_01 = getKlimited(binEdges,chiOfZ4,Fobj4,kmax=kmax1)\nprint 'starting linear P(k), kmax = {0}...'.format(kmax2)\ncovarBins4_02,dClVecsBins4_02,lmaxByBin4_02,preMaskBins4_02 = getKlimited(binEdges,chiOfZ4,Fobj4,kmax=kmax2)\n"

In [58]:
# compare preMaskBins to increasing binMax
#binMax = [2,3,4,5,6,99]
#for i,binNum in enumerate(binMax):
#    print binNum
#    print preMaskBins2_01[i]
#    #print preMaskBins2_02[i]
## that last bin is kappa only

In [59]:
print preMaskBins2_01.__len__(), preMaskBins2_02.__len__()
#print preMaskBins4_01.__len__(), preMaskBins4_02.__len__()

16 16


# Define function to find a set of kmax, zmax dependent constraints

In [60]:
# need to get:
# 2 piece from pmb[0]
# 3 piece from pmb[1] with 2,3 piece from pmb[0]
# 4 piece from pmb[2] with 4,3 piece from pmb[1] with 4,3,2 piece from pmb[0]
# ...
# 6 piece from pmb[4] with ... with 6,5,4,3,2 piece from pmb[0]

# do this with nested loop?
# loop1 over which block from pmb
#    loop2 over current and lower pieces

In [61]:
print covarBins2_01.__len__()
print dClVecsBins2_01.__len__()
print preMaskBins2_01.__len__()
print lmaxByBin2_01.__len__()
print FijTE2.shape

16
16
16
17
(9, 9)


In [62]:
# Define function to find a set of kmax, zmax dependent constraints
def getKZlimited(covarBins,dClVecsBins,preMaskBins,Fobj,lmaxByBin,FijTE,centers=True,
                 galaxiesOnly=False,fixW=False,addTE=True,onlyLastBlock=False,
                 getFij=False):
    """
    Purpose:
    
    Inputs:
      covarBins,dClVecsBins,preMaskBins: outputs from run of getKlimited: lists of arrays
      #Fij_kk: the kappa only piece for higher bins
      Fobj
      lmaxByBin: the ell value for each ell-block edge 
          There must be nBins+2 if centers==True, else nBins+1
      FijTE: the TE part to add in at the end
      nParams, nBins, paramList: like in FisherMatrix obj.
      centers: set to True to indicate that the values in binEdges are actually bin centers
          default: True
      galaxiesOnly: set to True to indicate that input has no kappa info (affects bias params)
          default: False
      fixW: set to True to fix the w0,wa parameters in all derived/returned results
          default: False
      addTE: set to True to add in the FijTE Fisher Matrix, False to omit adding it.
          default: True
      onlyLastBlock: set to True for extreme modification: should return what is needed
          for getFijSum call for last block only
          default: False
      getFij: set to True to also return list of Fisher matrices
          default: False
    Returns:
      list of arrays of sigmas, one list element for each of nBins-1 bins 
      (no bin1 because it's always cut off in current paradigm)
      FoMPropBins: list of FoM proportionality factors
      paramList (should be like input without bin1 param.)
      if getFij: list of Fisher matrices
    """
    nParams = Fobj.nParams
    nBins = Fobj.nBins
    nCosParams = nParams-nBins
    paramList = Fobj.paramList
    print 'lmaxByBin: ',lmaxByBin
    
    # check for exceptional case
    nBlocks = covarBins.__len__()
    #if nBlocks != nBins: # these are equal in the nominal case where lmax(kmax) is always below system lmax
    #    print 'nBlocks = ',nBlocks,'nBins = ',nBins
    #    print 'warning! check nBlocks and nBins!'
        
    # go through bins to show affect of increasing #bins/increasing redshift on param constraints
    # this binNum1 will effectively be the binNumMax in creating Fij
    # start from bin2 since that is the first place where there is a block to count
    FijBins = []
    if centers:
        sigmaBins = np.zeros((nBins,nParams))  # no bins lost for centers
        FoMPropBins = np.zeros(nBins)
        #if fixW:
        #    FijBins = np.zeros((nBins,nParams-2,nParams-2))
        #else:
        #    FijBins = np.zeros((nBins,nParams,nParams))
        minBinNum = 1
    else:
        sigmaBins = np.zeros((nBins-1,nParams-1))  # the -1s since we lose 1 bin and 1 bin bias param
        FoMPropBins = np.zeros(nBins-1)
        #if fixW:
        #    FijBins = np.zeros((nBins-1,nParams-2,nParams-2))
        #else:
        #    FijBins = np.zeros((nBins-1,nParams,nParams))
        minBinNum = 2
    # loop through bins to create redshift dependence
    if onlyLastBlock:
        minBinNum = nBins
    for binNum1 in range(minBinNum,nBins+1):
        sbIndex = binNum1-minBinNum # for indexing sigmaBins
        preMask = preMaskBins[sbIndex] # 
        print '\nstarting bin number ',binNum1%(nBins+1),', with sigmaBins index = ',sbIndex, \
              ', pmb[{0}] = {1}'.format(binNum1-minBinNum,preMask)
        
        # get contribution from all blocks available for each bin
        # count up to one less than binNum1
        myDClVecsBins = []
        myInvCovBins = []
        for blockCounter in range(1,binNum1): # does this line need to change for centers?
            print '  starting block ',blockCounter
            myBlockNum = binNum1-blockCounter-1 # to index input lists
            myBinNums = binNum1 - np.arange(blockCounter)
            myBinNums = np.append([0],myBinNums) # get kappa in there
            print '  use {0} bin numbers from pmb[{1}]'.format(blockCounter+1,myBlockNum), \
                  ' with bin numbers {0}'.format(myBinNums%(nBins+1))
        
            # get Fij for each piece:
            
            # get the covar and dClVec and preMask to work with
            myCovar = covarBins[myBlockNum].copy()
            myDClVecs = dClVecsBins[myBlockNum].copy()
            myPreMask = preMaskBins[myBlockNum].copy()
            #print 'covar.shape: ',myCovar.shape
            #print 'dClVecs.shape: ',myDClVecs.shape
            #print 'preMask.shape: ',myPreMask.shape
            

            # get a list of bin numbers to remove from preMask, covar, dClVec
            # remove all except myBinNums%(nBins+1)
            toCut = np.setdiff1d(np.arange(nBins+1),myBinNums%(nBins+1))
            print '  toCut: ',toCut
            

            # remove them using covarCut
            for cutNum in toCut:
                myCovar,myDClVecs,myPreMask = covarCut(myCovar,myDClVecs,myPreMask,cutNum,verbose=False)
            print '  the cut preMask: ',myPreMask
            #print '  the cut covar.shape: ',myCovar.shape
            #print '  the cut dClVecs.shape: ',myDClVecs.shape
            myDClVecsBins.append( myDClVecs )
            myInvCovBins.append( np.transpose(np.linalg.inv(np.transpose(myCovar)) ) )
            
        # get out if onlyLastBlock
        if onlyLastBlock and not getFij:
            return nParams,nBins,myDClVecsBins,myInvCovBins,paramList
            
        # add up the pieces into one Fisher Matrix    
        myFij,myParamList = getFijSum(nParams,nBins,myDClVecsBins,myInvCovBins,paramList,centers=centers)    
        print 'original myFij.shape: ',myFij.shape # bin1 bias already removed
        
        # trim rows,cols from unused bins
        toTrim = nBins-binNum1
        if toTrim > 0: # equal for bin6
            myFij = myFij[:-toTrim,:-toTrim]
        print 'trimmed myFij.shape: ',myFij.shape
        
        # add in kappa-only for higher ell
        Fij_kgTE = myFij.copy()
        if not galaxiesOnly:
            print 'for clkk: lmin = ',lmaxByBin[binNum1-1],', lmax = ,',lmaxByBin[-1]
            Fijkk = getFijkk(Fobj,myLmin=lmaxByBin[binNum1-1],lmax=lmaxByBin[-1],verbose=False)
            print 'Fijkk.shape: ',Fijkk.shape
            Fij_kgTE[:nCosParams,:nCosParams] += Fijkk
        
        # adjust As
        AsIndex = 3
        mult = 1e-9
        Fij_kgTE = rowColMult(Fij_kgTE,AsIndex,mult)
        
        # add in TE  # has to be after adjust As since FijTE has already had this transformation
        #Fij_kgTE = myFij.copy()
        if addTE:
            Fij_kgTE[:nCosParams,:nCosParams] += FijTE
        
        # set this to fix w0,wa
        if fixW:
            Fij_kgTE = fixParam(Fij_kgTE,8)
            Fij_kgTE = fixParam(Fij_kgTE,7)
        
        # get out if onlyLastBlock
        if onlyLastBlock and getFij:
            return nParams,nBins,myDClVecsBins,myInvCovBins,paramList, Fij_kgTE
        
        # add to Fij list (array)
        #FijBins[sbIndex] = Fij_kgTE
        FijBins.append(Fij_kgTE)
        
        # get sigmas and append
        sigmaBins[sbIndex][:Fij_kgTE.shape[0]] = getSigmas(Fij_kgTE)
        # try with fixed biases
        #sigmaBins[sbIndex][:nCosParams] = getSigmas(Fij_kgTE[:nCosParams,:nCosParams])
        
        
        # get FoM prop and append
        if not fixW:
            FijInv = np.linalg.inv(Fij_kgTE)
            FoMPropBins[sbIndex] = FoMprop(FijInv,iw0=7,iwa=8)
                                
    if getFij:
        return sigmaBins,FoMPropBins,myParamList,FijBins
    else:
        return sigmaBins,FoMPropBins,myParamList #the last paramList should be the biggest
            
# test it



#print nParams, nBins, '\n'
#sigmaBins2_01,FoMPropBins2_01,paramList2_01 = getKZlimited(covarBins2_01,dClVecsBins2_01,preMaskBins2_01,Fobj2,
#                                                           lmaxByBin2_01,FijTE2)
#for i in range(nBins-1):
#    print i,' : ',sigmaBins2_01[i]
#print 'FoM prop: ',FoMPropBins2_01


In [63]:
#sigmaBins2_01,FoMPropBins2_01,paramList2_01 = getKZlimited(covarBins2_01,dClVecsBins2_01,
#                                                           preMaskBins2_01,Fobj2,lmaxByBin2_01,FijTE2,
#                                                           centers=centers)
#sigmaBins2_02,FoMPropBins2_02,paramList2_02 = getKZlimited(covarBins2_02,dClVecsBins2_02,
#                                                           preMaskBins2_02,Fobj2,lmaxByBin2_02,FijTE2,
#                                                           centers=centers)
sigmaBins2_01,FoMPropBins2_01,paramList2_01,FijBins2_01 = getKZlimited(covarBins2_01,dClVecsBins2_01,
                                                           preMaskBins2_01,Fobj2,lmaxByBin2_01,FijTE2,
                                                           centers=centers,getFij=True)
sigmaBins2_02,FoMPropBins2_02,paramList2_02,FijBins2_02 = getKZlimited(covarBins2_02,dClVecsBins2_02,
                                                           preMaskBins2_02,Fobj2,lmaxByBin2_02,FijTE2,
                                                           centers=centers,getFij=True)

##sigmaBins4_01,FoMPropBins4_01,paramList4_01 = getKZlimited(covarBins4_01,dClVecsBins4_01,
##                                                           preMaskBins4_01,Fobj4,lmaxByBin4_01,FijTE4,
##                                                           centers=centers)
##sigmaBins4_02,FoMPropBins4_02,paramList4_02 = getKZlimited(covarBins4_02,dClVecsBins4_02,
##                                                           preMaskBins4_02,Fobj4,lmaxByBin4_02,FijTE4,
##                                                           centers=centers)
#sigmaBins4_01,FoMPropBins4_01,paramList4_01,FijBins4_01 = getKZlimited(covarBins4_01,dClVecsBins4_01,
#                                                           preMaskBins4_01,Fobj4,lmaxByBin4_01,FijTE4,
#                                                           centers=centers,getFij=True)
#sigmaBins4_02,FoMPropBins4_02,paramList4_02,FijBins4_02 = getKZlimited(covarBins4_02,dClVecsBins4_02,
#                                                           preMaskBins4_02,Fobj4,lmaxByBin4_02,FijTE4,
#                                                           centers=centers,getFij=True)
print 'done with all 2 cases.'

lmaxByBin:  [   2   84  160  226  285  337  384  425  462  496  526  567  603  644
  689  726 2000]

starting bin number  2 , with sigmaBins index =  0 , pmb[0] = [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  4  4  4
  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5  6  6
  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8  8  8  8
  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11 11 11 11
 11 12 12 12 12 12 13 13 13 13 14 14 14 15 15 16]
  starting block  1
  use 2 bin numbers from pmb[0]  with bin numbers [0 2]
  toCut:  [ 1  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  the cut preMask:  [0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
nBlocks:  1 , nBlocks2:  1
0 22950 (17, 25, 54) (17, 17, 54)
['ombh2' 'omch2' 'cosmomc_theta' '10^9 As' 'ns' 'tau' 'mnu' 'w' 'wa'
 'bin2' 'bin3' 'bin4' 'bin5' 'bin6' 'bin7' 'bin8' 'bin9' 'bin10' 'bin11'
 'bin12' 'bin13' 'bin14' 'bin1

  the cut preMask:  [0 4 5 6 7 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6
 6 6 6 6 7 7 7 7 7 7 7 7 7 7]
  starting block  5
  use 6 bin numbers from pmb[1]  with bin numbers [0 7 6 5 4 3]
  toCut:  [ 1  2  8  9 10 11 12 13 14 15 16]
  the cut preMask:  [0 3 4 5 6 7 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5
 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7]
  starting block  6
  use 7 bin numbers from pmb[0]  with bin numbers [0 7 6 5 4 3 2]
  toCut:  [ 1  8  9 10 11 12 13 14 15 16]
  the cut preMask:  [0 2 3 4 5 6 7 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4
 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 7 7
 7 7 7 7 7 7 7 7]
nBlocks:  6 , nBlocks2:  6
0 14100 (12, 25, 47) (12, 12, 47)
1 31200 (24, 25, 52) (24, 24, 52)
2 54575 (37, 25, 59) (37, 37, 59)
3 84150 (51, 25, 66) (51, 51, 66)
4 125400 (66, 25, 76) (66, 66, 76)
5 110700 (82, 25, 54) (82, 82, 54)
['ombh2' 'omch2' 'cosmomc_theta' '10^9 As'

  the cut preMask:  [ 0  4  5  6  7  8  9 10  4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5
  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7
  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10
 10 10 10 10 10 10]
  starting block  8
  use 9 bin numbers from pmb[1]  with bin numbers [ 0 10  9  8  7  6  5  4  3]
  toCut:  [ 1  2 11 12 13 14 15 16]
  the cut preMask:  [ 0  3  4  5  6  7  8  9 10  3  3  3  3  3  3  3  3  3  3  3  3  3  3  4
  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5
  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8  8
  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10]
  starting block  9
  use 10 bin numbers from pmb[0]  with bin numbers [ 0 10  9  8  7  6  5  4  3  2]
  toCut:  [ 1 11 12 13 14 15 16]
  the cut preMask:  [ 0  2  3  4  5  6  7  8  9 10  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4


  toCut:  [ 1  2 13 14 15 16]
  the cut preMask:  [ 0  3  4  5  6  7  8  9 10 11 12  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5
  5  5  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8
  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11
 11 11 11 11 11 12 12 12 12 12]
  starting block  11
  use 12 bin numbers from pmb[0]  with bin numbers [ 0 12 11 10  9  8  7  6  5  4  3  2]
  toCut:  [ 1 13 14 15 16]
  the cut preMask:  [ 0  2  3  4  5  6  7  8  9 10 11 12  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4
  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6
  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8
  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12
 12 12]
nBlocks:  11 , nBlocks2:  11
0 7175 (7, 25, 41) (7, 7, 41)
1 10500 (14, 25, 30) (14, 14, 30)
2 1

  toCut:  [ 1  2  3  4 15 16]
  the cut preMask:  [ 0  5  6  7  8  9 10 11 12 13 14  5  5  5  5  5  5  5  5  5  5  5  5  6
  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8  8  8
  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11 11 11
 11 11 12 12 12 12 12 13 13 13 13 14 14 14]
  starting block  11
  use 12 bin numbers from pmb[2]  with bin numbers [ 0 14 13 12 11 10  9  8  7  6  5  4]
  toCut:  [ 1  2  3 15 16]
  the cut preMask:  [ 0  4  5  6  7  8  9 10 11 12 13 14  4  4  4  4  4  4  4  4  4  4  4  4
  4  5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6
  7  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9  9  9
  9  9  9 10 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12 12 12 13 13 13
 13 14 14 14]
  starting block  12
  use 13 bin numbers from pmb[1]  with bin numbers [ 0 14 13 12 11 10  9  8  7  6  5  4  3]
  toCut:  [ 1  2 15 16]
  the cut preMask:  [ 0  3  4  5  6  7  8  9 10 11 12 13 14  3  3  3  3  3  3  3  3  3 

...

  starting block  3
  use 4 bin numbers from pmb[2]  with bin numbers [0 6 5 4]
  toCut:  [ 1  2  3  7  8  9 10 11 12 13 14 15 16]
  the cut preMask:  [0 4 5 6 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6
 6 6 6]
  starting block  4
  use 5 bin numbers from pmb[1]  with bin numbers [0 6 5 4 3]
  toCut:  [ 1  2  7  8  9 10 11 12 13 14 15 16]
  the cut preMask:  [0 3 4 5 6 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5
 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6]
  starting block  5
  use 6 bin numbers from pmb[0]  with bin numbers [0 6 5 4 3 2]
  toCut:  [ 1  7  8  9 10 11 12 13 14 15 16]
  the cut preMask:  [0 2 3 4 5 6 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4
 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6]
nBlocks:  5 , nBlocks2:  5
0 33800 (13, 25, 104) (13, 13, 104)
1 76700 (26, 25, 118) (26, 26, 118)
2 134000 (40, 25, 134) (40, 40, 134)
3 207625 (55, 25, 151) (55, 55, 151)
4 246725 (71, 25, 139) (71, 71, 139)

1 41500 (20, 25, 83) (20, 20, 83)
2 71300 (31, 25, 92) (31, 31, 92)
3 111800 (43, 25, 104) (43, 43, 104)
4 165200 (56, 25, 118) (56, 56, 118)
5 234500 (70, 25, 134) (70, 70, 134)
6 320875 (85, 25, 151) (85, 85, 151)
7 350975 (101, 25, 139) (101, 101, 139)
['ombh2' 'omch2' 'cosmomc_theta' '10^9 As' 'ns' 'tau' 'mnu' 'w' 'wa'
 'bin2' 'bin3' 'bin4' 'bin5' 'bin6' 'bin7' 'bin8' 'bin9' 'bin10' 'bin11'
 'bin12' 'bin13' 'bin14' 'bin15' 'bin16']
original myFij.shape:  (24, 24)
trimmed myFij.shape:  (17, 17)
for clkk: lmin =  925 , lmax = , 2000
Fijkk.shape:  (9, 9)

starting bin number  10 , with sigmaBins index =  8 , pmb[8] = [ 0 10 11 12 13 14 15 16 10 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12
 12 12 13 13 13 13 14 14 14 15 15 16]
  starting block  1
  use 2 bin numbers from pmb[8]  with bin numbers [ 0 10]
  toCut:  [ 1  2  3  4  5  6  7  8  9 11 12 13 14 15 16]
  the cut preMask:  [ 0 10 10 10 10 10 10 10 10]
  starting block  2
  use 3 bin numbers from pmb[7]  with bin numbers [ 0 10  9

  starting block  7
  use 8 bin numbers from pmb[4]  with bin numbers [ 0 12 11 10  9  8  7  6]
  toCut:  [ 1  2  3  4  5 13 14 15 16]
  the cut preMask:  [ 0  6  7  8  9 10 11 12  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7
  7  7  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10
 10 10 10 10 10 11 11 11 11 11 11 12 12 12 12 12]
  starting block  8
  use 9 bin numbers from pmb[3]  with bin numbers [ 0 12 11 10  9  8  7  6  5]
  toCut:  [ 1  2  3  4 13 14 15 16]
  the cut preMask:  [ 0  5  6  7  8  9 10 11 12  5  5  5  5  5  5  5  5  5  5  5  5  6  6  6
  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8
  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11 11 11 11 11
 12 12 12 12 12]
  starting block  9
  use 10 bin numbers from pmb[2]  with bin numbers [ 0 12 11 10  9  8  7  6  5  4]
  toCut:  [ 1  2  3 13 14 15 16]
  the cut preMask:  [ 0  4  5  6  7  8  9 10 11 12  4  4  4  4  4  4  4  4  4  4  4  4  4  5
  5  5  5  5  5  5  5  5  5  

  the cut preMask:  [ 0  7  8  9 10 11 12 13 14  7  7  7  7  7  7  7  7  7  7  8  8  8  8  8
  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11 11 11 11
 11 12 12 12 12 12 13 13 13 13 14 14 14]
  starting block  9
  use 10 bin numbers from pmb[4]  with bin numbers [ 0 14 13 12 11 10  9  8  7  6]
  toCut:  [ 1  2  3  4  5 15 16]
  the cut preMask:  [ 0  6  7  8  9 10 11 12 13 14  6  6  6  6  6  6  6  6  6  6  6  7  7  7
  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9
 10 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12 12 12 13 13 13 13 14 14
 14]
  starting block  10
  use 11 bin numbers from pmb[3]  with bin numbers [ 0 14 13 12 11 10  9  8  7  6  5]
  toCut:  [ 1  2  3  4 15 16]
  the cut preMask:  [ 0  5  6  7  8  9 10 11 12 13 14  5  5  5  5  5  5  5  5  5  5  5  5  6
  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8  8  8
  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11 11 11
 11 11 12 12 12 12 12 13 13 13 13 14 

5 56950 (34, 25, 67) (34, 34, 67)
6 79550 (43, 25, 74) (43, 43, 74)
7 109975 (53, 25, 83) (53, 53, 83)
8 147200 (64, 25, 92) (64, 64, 92)
9 197600 (76, 25, 104) (76, 76, 104)
10 262550 (89, 25, 118) (89, 89, 118)
11 345050 (103, 25, 134) (103, 103, 134)
12 445450 (118, 25, 151) (118, 118, 151)
13 465650 (134, 25, 139) (134, 134, 139)
['ombh2' 'omch2' 'cosmomc_theta' '10^9 As' 'ns' 'tau' 'mnu' 'w' 'wa'
 'bin2' 'bin3' 'bin4' 'bin5' 'bin6' 'bin7' 'bin8' 'bin9' 'bin10' 'bin11'
 'bin12' 'bin13' 'bin14' 'bin15' 'bin16']
original myFij.shape:  (24, 24)
trimmed myFij.shape:  (23, 23)
for clkk: lmin =  1378 , lmax = , 2000
Fijkk.shape:  (9, 9)

starting bin number  16 , with sigmaBins index =  14 , pmb[14] = [ 0 16 16]
  starting block  1
  use 2 bin numbers from pmb[14]  with bin numbers [ 0 16]
  toCut:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
  the cut preMask:  [ 0 16 16]
  starting block  2
  use 3 bin numbers from pmb[13]  with bin numbers [ 0 16 15]
  toCut:  [ 1  2  3  4  5  6  7

In [64]:
# compare Fij for last bin
#myNBins = FijBins2_02.__len__()
#print myNBins
#for binNum in range(myNBins):
#    print FijBins2_02[binNum].shape
#myNBins = FijBins4_02.__len__()
#print myNBins
#for binNum in range(myNBins):
#    print FijBins4_02[binNum].shape

In [65]:
#print Fij_kgTE2_02
#print FijBins2_02[14]
#FDiff = Fij_kgTE2_02-FijBins2_02[14]
#FDMap = np.zeros((24,24))
#FDMap[np.where(FDiff != 0)] = 1
#print FDMap

In [66]:
# compare sigmas - biases fixed
#sigmas_t1 = getSigmas(Fij_kgTE2_02[:9,:9])
#sigmas_t2 = getSigmas(FijBins2_02[14][:9,:9])
#print 'biases fixed'
#for i,paramName in enumerate(paramList[:9]):
#    print paramName,sigmas_t1[i],sigmas_t2[i]

In [67]:
print paramList

['ombh2', 'omch2', 'cosmomc_theta', '10^9 As', 'ns', 'tau', 'mnu', 'w', 'wa', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6', 'bin7', 'bin8', 'bin9', 'bin10', 'bin11', 'bin12', 'bin13', 'bin14', 'bin15', 'bin16']


In [68]:
print 'compare sigmas for last bin'
print 'fsky=1'
print '{0} P(k)'.format(label2)
#Fijt3 = fixParam(Fij_kgTE2_02,9)
#Fijt4 = fixParam(FijBins2_02[14],9)
paramList_t3 = np.delete(paramList,9)
sigmas_t3 = getSigmas(Fij_kgTE2_02)
sigmas_t4 = getSigmas(FijBins2_02[14])
print 'biases marginalized'
for i,paramName in enumerate(paramList_t3):
    print paramName,sigmas_t3[i],sigmas_t4[i],sigmaBins2_02[14][i]

compare sigmas for last bin
fsky=1
linear P(k)
biases marginalized
ombh2 8.223925744940824e-06 8.223925744940918e-06 8.223925744940918e-06
omch2 0.00024510590051423953 0.0002451059005142781 0.0002451059005142781
cosmomc_theta 3.2208339205269467e-07 3.2208339205271526e-07 3.2208339205271526e-07
10^9 As 0.017706497696945895 0.01770649769695065 0.01770649769695065
ns 0.0009727351029969976 0.0009727351029970479 0.0009727351029970479
tau 0.004463955653048506 0.004463955653049694 0.004463955653049694
mnu 0.022875508169052743 0.022875508169054266 0.022875508169054266
w 0.13837820868463518 0.13837820868467907 0.13837820868467907
wa 0.3057551514445617 0.3057551514446627 0.3057551514446627
bin2 0.010479550676567086 0.010479550676568049 0.010479550676568049
bin3 0.006760827837659007 0.006760827837659199 0.006760827837659199
bin4 0.005670928194276462 0.0056709281942767505 0.0056709281942767505
bin5 0.005626911278266474 0.005626911278267137 0.005626911278267137
bin6 0.005771147159294946 0.005771147

In [69]:
# fix w,wa
print 'compare sigmas for last bin'
print 'fsky=1'
print '{0} P(k)'.format(label2)

Fijt3 = fixParam(Fij_kgTE2_02,8)
Fijt3 = fixParam(Fijt3,7)
Fijt4 = fixParam(FijBins2_02[14],8)
Fijt4 = fixParam(Fijt4,7)
paramList_t4 = np.delete(paramList_t3,8)
paramList_t4 = np.delete(paramList_t4,7)
sigmas_t3 = getSigmas(Fijt3)
sigmas_t4 = getSigmas(Fijt4)
print 'biases marginalized; w0,wa fixed'
for i,paramName in enumerate(paramList_t4):
    print paramName,sigmas_t3[i],sigmas_t4[i]

compare sigmas for last bin
fsky=1
linear P(k)
biases marginalized; w0,wa fixed
ombh2 8.190603273530024e-06 8.190603273530136e-06
omch2 0.00020731026818203137 0.00020731026818203275
cosmomc_theta 2.782205655103509e-07 2.782205655103565e-07
10^9 As 0.01534426550773902 0.015344265507741003
ns 0.0009520312970452006 0.0009520312970452071
tau 0.0038895634130397608 0.0038895634130402352
mnu 0.020294757447385334 0.02029475744738808
bin2 0.006771806649453657 0.006771806649453892
bin3 0.005446377971354646 0.005446377971354984
bin4 0.005311228194216475 0.005311228194216864
bin5 0.005427182435698302 0.005427182435698723
bin6 0.005594049273626304 0.00559404927362674
bin7 0.005781724827735175 0.005781724827735634
bin8 0.006053102014652584 0.006053102014653069
bin9 0.0062234525891772426 0.006223452589177729
bin10 0.006408755593948055 0.0064087555939485435
bin11 0.00672072731716524 0.006720727317165739
bin12 0.007150917390436146 0.007150917390436644
bin13 0.007679432376579601 0.007679432376580078
bin

In [70]:
# same with fsky
print 'compare sigmas for last bin'
print 'fsky changed...'
print '{0} P(k)'.format(label2)

fsky = 0.44
skyfac = fsky**(-.5)
print 'biases marginalized; w0,wa fixed, fsky = {0}'.format(fsky)
for i,paramName in enumerate(paramList_t4):
    print paramName,sigmas_t3[i]*skyfac,sigmas_t4[i]*skyfac

compare sigmas for last bin
fsky changed...
linear P(k)
biases marginalized; w0,wa fixed, fsky = 0.44
ombh2 1.2347799029525349e-05 1.2347799029525518e-05
omch2 0.0003125319885217052 0.0003125319885217073
cosmomc_theta 4.194332839810583e-07 4.194332839810668e-07
10^9 As 0.023132350623982962 0.023132350623985953
ns 0.0014352411822610535 0.0014352411822610633
tau 0.005863737472430468 0.0058637374724311835
mnu 0.030595498029203666 0.03059549802920781
bin2 0.010208882640487062 0.010208882640487416
bin3 0.00821072372610926 0.00821072372610977
bin4 0.008006977770987684 0.00800697777098827
bin5 0.008181785367281086 0.00818178536728172
bin6 0.008433346590626644 0.008433346590627302
bin7 0.008716278133945356 0.008716278133946048
bin8 0.009125394636521351 0.009125394636522084
bin9 0.009382207790393973 0.009382207790394706
bin10 0.009661562581007711 0.009661562581008446
bin11 0.010131877649694988 0.010131877649695739
bin12 0.010780413586774574 0.010780413586775325
bin13 0.011577179907282631 0.0115

In [71]:
# now for galaxies only case
#sigmaBins2_01g,FoMPropBins2_01g,paramList2_01g = getKZlimited(covarBins2_01g,dClVecsBins2_01g,
#                                                           preMaskBins2_01g,Fobj2,lmaxByBin2_01g,FijTE2,
#                                                           centers=centers,galaxiesOnly=True)
#sigmaBins2_02g,FoMPropBins2_02g,paramList2_02g = getKZlimited(covarBins2_02g,dClVecsBins2_02g,
#                                                           preMaskBins2_02g,Fobj2,lmaxByBin2_02g,FijTE2,
#                                                           centers=centers,galaxiesOnly=True)
sigmaBins2_01g,FoMPropBins2_01g,paramList2_01g,FijBins2_01g = getKZlimited(covarBins2_01g,dClVecsBins2_01g,
                                                           preMaskBins2_01g,Fobj2,lmaxByBin2_01g,FijTE2,
                                                           centers=centers,galaxiesOnly=True,getFij=True)
sigmaBins2_02g,FoMPropBins2_02g,paramList2_02g,FijBins2_02g = getKZlimited(covarBins2_02g,dClVecsBins2_02g,
                                                           preMaskBins2_02g,Fobj2,lmaxByBin2_02g,FijTE2,
                                                           centers=centers,galaxiesOnly=True,getFij=True)

##sigmaBins4_01g,FoMPropBins4_01g,paramList4_01g = getKZlimited(covarBins4_01g,dClVecsBins4_01g,
##                                                           preMaskBins4_01g,Fobj4,lmaxByBin4_01g,FijTE4,
##                                                           centers=centers,galaxiesOnly=True)
##sigmaBins4_02g,FoMPropBins4_02g,paramList4_02g = getKZlimited(covarBins4_02g,dClVecsBins4_02g,
##                                                           preMaskBins4_02g,Fobj4,lmaxByBin4_02g,FijTE4,
##                                                           centers=centers,galaxiesOnly=True)
#sigmaBins4_01g,FoMPropBins4_01g,paramList4_01g,FijBins4_01g = getKZlimited(covarBins4_01g,dClVecsBins4_01g,
#                                                           preMaskBins4_01g,Fobj4,lmaxByBin4_01g,FijTE4,
#                                                           centers=centers,galaxiesOnly=True,getFij=True)
#sigmaBins4_02g,FoMPropBins4_02g,paramList4_02g,FijBins4_02g = getKZlimited(covarBins4_02g,dClVecsBins4_02g,
#                                                           preMaskBins4_02g,Fobj4,lmaxByBin4_02g,FijTE4,
#                                                           centers=centers,galaxiesOnly=True,getFij=True)

print 'done with all 2 cases.'

lmaxByBin:  [   2   84  160  226  285  337  384  425  462  496  526  567  603  644
  689  726 2000]

starting bin number  2 , with sigmaBins index =  0 , pmb[0] = [ 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5
  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7
  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10
 10 10 10 11 11 11 11 11 11 12 12 12 12 12 13 13 13 13 14 14 14 15 15 16]
  starting block  1
  use 2 bin numbers from pmb[0]  with bin numbers [0 2]
  toCut:  [ 1  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  the cut preMask:  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
nBlocks:  1 , nBlocks2:  1
0 20250 (15, 25, 54) (15, 15, 54)
['ombh2' 'omch2' 'cosmomc_theta' '10^9 As' 'ns' 'tau' 'mnu' 'w' 'wa'
 'bin2' 'bin3' 'bin4' 'bin5' 'bin6' 'bin7' 'bin8' 'bin9' 'bin10' 'bin11'
 'bin12' 'bin13' 'bin14' 'bin15' 'bin16']
original myFij.shape:  (24, 24)
trimmed m

  the cut preMask:  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4
 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7
 7]
nBlocks:  6 , nBlocks2:  6
0 11750 (10, 25, 47) (10, 10, 47)
1 27300 (21, 25, 52) (21, 21, 52)
2 48675 (33, 25, 59) (33, 33, 59)
3 75900 (46, 25, 66) (46, 46, 66)
4 114000 (60, 25, 76) (60, 60, 76)
5 101250 (75, 25, 54) (75, 75, 54)
['ombh2' 'omch2' 'cosmomc_theta' '10^9 As' 'ns' 'tau' 'mnu' 'w' 'wa'
 'bin2' 'bin3' 'bin4' 'bin5' 'bin6' 'bin7' 'bin8' 'bin9' 'bin10' 'bin11'
 'bin12' 'bin13' 'bin14' 'bin15' 'bin16']
original myFij.shape:  (24, 24)
trimmed myFij.shape:  (15, 15)

starting bin number  8 , with sigmaBins index =  6 , pmb[6] = [ 8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10
 11 11 11 11 11 11 12 12 12 12 12 13 13 13 13 14 14 14 15 15 16]
  starting block  1
  use 2 bin numbers from pmb[6]  with bin numbers [0 8]
  toCut:  [ 1  2  3  4  5  6  7  9 10 11 12 13 14 15 16]
  the cut pr

  the cut preMask:  [ 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5
  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7
  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10
 10 10 10]
nBlocks:  9 , nBlocks2:  9
0 5950 (7, 25, 34) (7, 7, 34)
1 13875 (15, 25, 37) (15, 15, 37)
2 24600 (24, 25, 41) (24, 24, 41)
3 39950 (34, 25, 47) (34, 34, 47)
4 58500 (45, 25, 52) (45, 45, 52)
5 84075 (57, 25, 59) (57, 57, 59)
6 115500 (70, 25, 66) (70, 70, 66)
7 159600 (84, 25, 76) (84, 84, 76)
8 133650 (99, 25, 54) (99, 99, 54)
['ombh2' 'omch2' 'cosmomc_theta' '10^9 As' 'ns' 'tau' 'mnu' 'w' 'wa'
 'bin2' 'bin3' 'bin4' 'bin5' 'bin6' 'bin7' 'bin8' 'bin9' 'bin10' 'bin11'
 'bin12' 'bin13' 'bin14' 'bin15' 'bin16']
original myFij.shape:  (24, 24)
trimmed myFij.shape:  (18, 18)

starting bin number  11 , with sigmaBins index =  9 , pmb[9] = [11 11 11 11 11 11 12 12 12 12 12 

1 8250 (11, 25, 30) (11, 11, 30)
2 15300 (18, 25, 34) (18, 18, 34)
3 24050 (26, 25, 37) (26, 26, 37)
4 35875 (35, 25, 41) (35, 35, 41)
5 52875 (45, 25, 47) (45, 45, 47)
6 72800 (56, 25, 52) (56, 56, 52)
7 100300 (68, 25, 59) (68, 68, 59)
8 133650 (81, 25, 66) (81, 81, 66)
9 180500 (95, 25, 76) (95, 95, 76)
10 148500 (110, 25, 54) (110, 110, 54)
['ombh2' 'omch2' 'cosmomc_theta' '10^9 As' 'ns' 'tau' 'mnu' 'w' 'wa'
 'bin2' 'bin3' 'bin4' 'bin5' 'bin6' 'bin7' 'bin8' 'bin9' 'bin10' 'bin11'
 'bin12' 'bin13' 'bin14' 'bin15' 'bin16']
original myFij.shape:  (24, 24)
trimmed myFij.shape:  (20, 20)

starting bin number  13 , with sigmaBins index =  11 , pmb[11] = [13 13 13 13 14 14 14 15 15 16]
  starting block  1
  use 2 bin numbers from pmb[11]  with bin numbers [ 0 13]
  toCut:  [ 1  2  3  4  5  6  7  8  9 10 11 12 14 15 16]
  the cut preMask:  [13 13 13 13]
  starting block  2
  use 3 bin numbers from pmb[10]  with bin numbers [ 0 13 12]
  toCut:  [ 1  2  3  4  5  6  7  8  9 10 11 14 15 16]
  

  starting block  13
  use 14 bin numbers from pmb[0]  with bin numbers [ 0 14 13 12 11 10  9  8  7  6  5  4  3  2]
  toCut:  [ 1 15 16]
  the cut preMask:  [ 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5
  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7
  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10
 10 10 10 11 11 11 11 11 11 12 12 12 12 12 13 13 13 13 14 14 14]
nBlocks:  13 , nBlocks2:  13
0 3075 (3, 25, 41) (3, 3, 41)
1 6300 (7, 25, 36) (7, 7, 36)
2 12300 (12, 25, 41) (12, 12, 41)
3 13500 (18, 25, 30) (18, 18, 30)
4 21250 (25, 25, 34) (25, 25, 34)
5 30525 (33, 25, 37) (33, 33, 37)
6 43050 (42, 25, 41) (42, 42, 41)
7 61100 (52, 25, 47) (52, 52, 47)
8 81900 (63, 25, 52) (63, 63, 52)
9 110625 (75, 25, 59) (75, 75, 59)
10 145200 (88, 25, 66) (88, 88, 66)
11 193800 (102, 25, 76) (102, 102, 76)
12 157950 (117, 25, 54) (117, 117, 54)
['ombh2' '

...

  starting block  5
  use 6 bin numbers from pmb[0]  with bin numbers [0 6 5 4 3 2]
  toCut:  [ 1  7  8  9 10 11 12 13 14 15 16]
  the cut preMask:  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4
 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6]
nBlocks:  5 , nBlocks2:  5
0 28600 (11, 25, 104) (11, 11, 104)
1 67850 (23, 25, 118) (23, 23, 118)
2 120600 (36, 25, 134) (36, 36, 134)
3 188750 (50, 25, 151) (50, 50, 151)
4 225875 (65, 25, 139) (65, 65, 139)
['ombh2' 'omch2' 'cosmomc_theta' '10^9 As' 'ns' 'tau' 'mnu' 'w' 'wa'
 'bin2' 'bin3' 'bin4' 'bin5' 'bin6' 'bin7' 'bin8' 'bin9' 'bin10' 'bin11'
 'bin12' 'bin13' 'bin14' 'bin15' 'bin16']
original myFij.shape:  (24, 24)
trimmed myFij.shape:  (14, 14)

starting bin number  7 , with sigmaBins index =  5 , pmb[5] = [ 7  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9  9  9
  9  9  9 10 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12 12 12 13 13 13
 13 14 14 14 15 15 16]
  starting block  1
  use 2 b

  starting block  6
  use 7 bin numbers from pmb[3]  with bin numbers [ 0 10  9  8  7  6  5]
  toCut:  [ 1  2  3  4 11 12 13 14 15 16]
  the cut preMask:  [ 5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6  7
  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9
  9  9 10 10 10 10 10 10 10]
  starting block  7
  use 8 bin numbers from pmb[2]  with bin numbers [ 0 10  9  8  7  6  5  4]
  toCut:  [ 1  2  3 11 12 13 14 15 16]
  the cut preMask:  [ 4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5
  5  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10]
  starting block  8
  use 9 bin numbers from pmb[1]  with bin numbers [ 0 10  9  8  7  6  5  4  3]
  toCut:  [ 1  2 11 12 13 14 15 16]
  the cut preMask:  [ 3  3  3  3  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4
  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6


  the cut preMask:  [ 4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5
  5  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11
 11 11 11 11 12 12 12 12 12]
  starting block  10
  use 11 bin numbers from pmb[1]  with bin numbers [ 0 12 11 10  9  8  7  6  5  4  3]
  toCut:  [ 1  2 13 14 15 16]
  the cut preMask:  [ 3  3  3  3  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4
  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6
  6  6  7  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9
  9  9  9  9  9 10 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12 12 12]
  starting block  11
  use 12 bin numbers from pmb[0]  with bin numbers [ 0 12 11 10  9  8  7  6  5  4  3  2]
  toCut:  [ 1 13 14 15 16]
  the cut preMask:  [ 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  4  4  4 

  the cut preMask:  [ 6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8  8
  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11 11
 11 11 11 12 12 12 12 12 13 13 13 13 14 14 14]
  starting block  10
  use 11 bin numbers from pmb[3]  with bin numbers [ 0 14 13 12 11 10  9  8  7  6  5]
  toCut:  [ 1  2  3  4 15 16]
  the cut preMask:  [ 5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6  7
  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9
  9  9 10 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12 12 12 13 13 13 13
 14 14 14]
  starting block  11
  use 12 bin numbers from pmb[2]  with bin numbers [ 0 14 13 12 11 10  9  8  7  6  5  4]
  toCut:  [ 1  2  3 15 16]
  the cut preMask:  [ 4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5
  5  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11
 11 11 11 11 12 12 12 12

  the cut preMask:  [ 6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8  8
  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11 11
 11 11 11 12 12 12 12 12 13 13 13 13 14 14 14 15 15 16]
  starting block  12
  use 13 bin numbers from pmb[3]  with bin numbers [ 0 16 15 14 13 12 11 10  9  8  7  6  5]
  toCut:  [1 2 3 4]
  the cut preMask:  [ 5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6  7
  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9
  9  9 10 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12 12 12 13 13 13 13
 14 14 14 15 15 16]
  starting block  13
  use 14 bin numbers from pmb[2]  with bin numbers [ 0 16 15 14 13 12 11 10  9  8  7  6  5  4]
  toCut:  [1 2 3]
  the cut preMask:  [ 4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5
  5  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11
 11 11 11 11 

In [72]:
# save the results
toSave5 = [sigmaBins2_01,FoMPropBins2_01,paramList2_01,FijBins2_01,sigmaBins2_01g,FoMPropBins2_01g,paramList2_01g,FijBins2_01g]
toSave6 = [sigmaBins2_02,FoMPropBins2_02,paramList2_02,FijBins2_02,sigmaBins2_02g,FoMPropBins2_02g,paramList2_02g,FijBins2_02g]
#toSave7 = [sigmaBins4_01,FoMPropBins4_01,paramList4_01,FijBins4_01,sigmaBins4_01g,FoMPropBins4_01g,paramList4_01g,FijBins4_01g]
#toSave8 = [sigmaBins4_02,FoMPropBins4_02,paramList4_02,FijBins4_02,sigmaBins4_02g,FoMPropBins4_02g,paramList4_02g,FijBins4_02g]

#np.save('KZlimited_gold_nln_k1_wa1.npy',toSave5)
#np.save('KZlimited_gold_nln_k2_wa1.npy',toSave6)
#np.save('KZlimited_opti_nln_k1_wa1.npy',toSave7)
#np.save('KZlimited_opti_nln_k2_wa1.npy',toSave8)

if fileNum == 1:
    np.save('KZlimited_gold_lin_k1_wa1_full_C.npy',toSave5)
    np.save('KZlimited_gold_lin_k2_wa1_full_C.npy',toSave6)
elif fileNum == 2:
    np.save('KZlimited_gold_lin_k1_wa1_shapeless_C.npy',toSave5)
    np.save('KZlimited_gold_lin_k2_wa1_shapeless_C.npy',toSave6)
elif fileNum == 3:
    np.save('KZlimited_gold_lin_k1_wa1_pzkfixed_C.npy',toSave5)
    np.save('KZlimited_gold_lin_k2_wa1_pzkfixed_C.npy',toSave6)
elif fileNum == 4:
    np.save('KZlimited_gold_lin_k1_wa1_chifixed_C.npy',toSave5)
    np.save('KZlimited_gold_lin_k2_wa1_chifixed_C.npy',toSave6)
elif fileNum == 5:
    np.save('KZlimited_gold_lin_k1_wa1_chiShapefixed_C.npy',toSave5)
    np.save('KZlimited_gold_lin_k2_wa1_chiShapefixed_C.npy',toSave6)

print 'done.'

done.
